In [1]:
import pandas as pd
import numpy as np
import requests
import os
from pandas_datareader import wb

In [2]:
countries = pd.read_csv('iso3_to_wb_name.csv',sep=',')
names_to_iso = pd.read_excel('names_to_iso.xlsx')
iso_to_wb = pd.read_csv('iso3_to_wb_name.csv')
iso_to_wb = iso_to_wb.set_index('iso3')
dict_map = {b:a for a,b in zip(names_to_iso.iso3,names_to_iso.country)}

In [3]:
def get_latest_year(data):
    """Get latest year of data"""
    df = pd.DataFrame()
    for country in data['Country Name'].unique():
        temp = data[data['Country Name']==country]
        temp = temp[temp.Year==temp.Year.max()]
        df = df.append(temp)
    return df.reset_index(drop=True)

def func(a,b):
    return (a - b) / (np.log(a) - np.log(b))

def map_country(x,dict_map):
    if x in dict_map.keys():
        return dict_map[x]
    else:
        return x

In [4]:
regions = ['Arab World', 'Caribbean small states','Central Europe and the Baltics', 'Early-demographic dividend','East Asia & Pacific','East Asia & Pacific (excluding high income)','East Asia & Pacific (IDA & IBRD countries)', 'Euro area','Europe & Central Asia',
'Europe & Central Asia (excluding high income)','Europe & Central Asia (IDA & IBRD countries)', 'European Union','Fragile and conflict affected situations',
'Heavily indebted poor countries (HIPC)', 'High income','IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only','IDA total', 'Late-demographic dividend','Latin America & Caribbean','Latin America & Caribbean (excluding high income)','Latin America & the Caribbean (IDA & IBRD countries)','Least developed countries: UN classification',
'Low & middle income', 'Low income', 'Lower middle income','Middle East & North Africa','Middle East & North Africa (excluding high income)','Middle East & North Africa (IDA & IBRD countries)','Middle income', 'North America', 'OECD members',
'Other small states', 'Pacific island small states','Post-demographic dividend', 'Pre-demographic dividend','Small states', 'South Asia', 'South Asia (IDA & IBRD)','Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
'Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income','World','East Asia & Pacific (all income levels)',
'Europe & Central Asia (all income levels)','Latin America & Caribbean (all income levels)','Middle East & North Africa (all income levels)',
'South Asia (all income levels)','Sub-Saharan Africa (all income levels)',]

## Emissions - CAIT - API

In [48]:
df_full = pd.DataFrame()
out = []
for iso in countries.iso3.unique():
    try:
        df_country = pd.DataFrame()
        r = requests.get('https://www.climatewatchdata.org/api/v1/data/historical_emissions?regions=%s'%iso)
        x = r.json()
        df = pd.DataFrame(x['data'])
        df = df[df.data_source=='CAIT']

        for i in df.index:
            temp = df[i:i+1]
            dict_em = temp['emissions'][i]
            num = len(dict_em)
            temp = temp.drop('emissions',axis=1)
            newdf = pd.DataFrame(np.repeat(temp.values,num,axis=0))
            newdf.columns = temp.columns
            newdf[['year','value']] = pd.DataFrame(dict_em)
            df_country = df_country.append(newdf)
        df_full = df_full.append(df_country)
    except:
        out.append(iso)
        
df_full = df_full.set_index('iso_code3')
df_full['country'] = iso_to_wb['country']
df_full.reset_index().to_excel('Data/Emissions/Emissions_CAIT.xlsx')

KeyError: "None of ['iso_code3'] are in the columns"

#### Contribution to total Growth

In [ ]:
df = pd.read_excel('Data/Emissions/Emissions_CAIT.xlsx')
df = df[df.gas=='All GHG']
df = df[df.sector.isin(['Industrial Processes', 'Agriculture', 'Waste',
       'Land-Use Change and Forestry', 'Electricity/Heat',
       'Manufacturing/Construction', 'Transportation', 'Building',
       'Other Fuel Combustion', 'Fugitive Emissions'])]

df_group = df.groupby(['iso_code3','country','gas','year','unit']).sum()
df_group = df_group.reset_index()

df_group['sector'] = 'Total (excluding Bunker Fuels)'

df = df.append(df_group)

In [ ]:
df_ = df[df.year.isin([2012,2018])]

df_2018 = df_[df_.year==2018]
df_2012 = df_[df_.year==2012]

df_2018 = df_2018.set_index(['country','sector'])
df_2012 = df_2012.set_index(['country','sector'])

df_2018['Sectoral Growth'] = (df_2018['value'] - df_2012['value'])/ df_2012['value']
df_2018['Sectoral diff'] = df_2018['value'] - df_2012['value']

df_2018 = df_2018.reset_index()

df_tot = pd.DataFrame()
for count in df_2018.country.unique():
    df_country = df_2018[df_2018.country==count]    
    diff_tot = df_country.loc[df_country.sector=='Total (excluding Bunker Fuels)', 'Sectoral diff'].values[0] 
    growth_tot = df_country.loc[df_country.sector=='Total (excluding Bunker Fuels)', 'Sectoral Growth'].values[0]
    df_country['Contribution_Total_Growth'] = growth_tot * df_country['Sectoral diff'] / diff_tot
    
    df_country.loc[df_country.sector=='Total (excluding Bunker Fuels)','Contribution_Total_Growth'] = -100
    df_country = df_country.sort_values('Contribution_Total_Growth',ascending=False)
    df_country['Ranking'] = range(1,len(df_country)+1)
    df_country.loc[df_country.sector=='Total (excluding Bunker Fuels)','Contribution_Total_Growth'] = 0
    
    df_tot = df_tot.append(df_country)

df_tot.to_excel('Data/Emissions/Contribution_Total_Growth.xlsx')

#### Contribution total growth (GDP, Pop)

In [103]:
df = pd.read_excel('Data/Emissions/Emissions_CAIT.xlsx')
df = df[df.gas=='All GHG']
df = df[df.sector.isin(['Industrial Processes', 'Agriculture', 'Waste','Land-Use Change and Forestry', 'Electricity/Heat',
       'Manufacturing/Construction', 'Transportation', 'Building','Other Fuel Combustion', 'Fugitive Emissions'])]


df_ = df[df.year.isin([2012,2018])]

df_2018 = df_[df_.year==2018]
df_2012 = df_[df_.year==2012]

df_2018 = df_2018.set_index(['country','sector'])
df_2012 = df_2012.set_index(['country','sector'])

df_tot = df_2018.append(df_2012)
df_tot = df_tot.reset_index()
df_tot = df_tot.set_index(['country','year'])

gdp = wb.download(indicator='NY.GDP.MKTP.CD',country='all',start=1990,end=2020)
gdp = gdp.reset_index().dropna()
gdp.columns = ['Country Name','Year','GDP']
gdp = gdp[~gdp['Country Name'].isin(regions)]
gdp['Year'] = gdp['Year'].astype(int)

gdp = gdp.set_index(['Country Name','Year'])
df_tot['GDP'] = gdp['GDP']

pop = wb.download(indicator='SP.POP.TOTL',country='all',start=1980,end=2020)
pop = pop.reset_index().dropna()
pop.columns = ['Country Name','Year','Population']
pop = pop[~pop['Country Name'].isin(regions)]
pop['Year'] = pop['Year'].astype(int)

pop = pop.set_index(['Country Name','Year'])

df_tot['Population'] = pop['Population']

df_tot = df_tot.reset_index()
df_tot['GDP_cap'] = df_tot['GDP'] / df_tot['Population']
df_tot['GHG_GDP'] = df_tot['value'] / df_tot['GDP']

In [104]:
df_out_total = pd.DataFrame()
for count in df_tot.country.unique():
    df_out = pd.DataFrame()
    df_country = df_tot[df_tot.country==count]
    a = np.where(df_country['value']<0)
    if len(a[0])>0:
        df_country = df_country[df_country.sector!='Land-Use Change and Forestry']
    ghg_tot_2018 = df_country.groupby('year').sum().loc[2018,'value']
    ghg_tot_2012 = df_country.groupby('year').sum().loc[2012,'value']
    L_GHG_tot = func(ghg_tot_2018,ghg_tot_2012)
    df_country_2018 = df_country[df_country.year==2018].set_index('sector')
    df_country_2012 = df_country[df_country.year==2012].set_index('sector')
    vect = (df_country_2018['value'] - df_country_2012['value']) / (np.log(df_country_2018['value']) - np.log(df_country_2012['value']))
#     D_Pop = np.exp(np.sum(vect.values) * np.log(df_country_2018.loc['Agriculture','Population']/df_country_2012.loc['Agriculture','Population'])/L_GHG_tot)
#     D_GDP = np.exp(np.sum(vect.values) * np.log(df_country_2018.loc['Agriculture','GDP_cap']/df_country_2012.loc['Agriculture','GDP_cap'])/L_GHG_tot)
#     D_GHG = np.exp(np.sum(vect * np.log(df_country_2018.loc[:,'GHG_GDP']/df_country_2012.loc[:,'GHG_GDP']))/L_GHG_tot)
    
    Diff_Pop = np.nansum(vect.values) * np.log(df_country_2018.loc['Agriculture','Population']/df_country_2012.loc['Agriculture','Population'])
    Diff_GDP = np.nansum(vect.values) * np.log(df_country_2018.loc['Agriculture','GDP_cap']/df_country_2012.loc['Agriculture','GDP_cap'])
    Diff_GHG = np.nansum(vect * np.log(df_country_2018.loc[:,'GHG_GDP']/df_country_2012.loc[:,'GHG_GDP']))
    
    df_out['Indicator'] = ['GDP_cap','Pop','GHG_GDP'] 
    df_out['Value'] = [Diff_GDP,Diff_Pop,Diff_GHG] / ghg_tot_2012
    df_out['Country Name'] = count
    
    df_out_total = df_out_total.append(df_out)

df_out_total_gr = df_out_total.groupby('Country Name').sum()
df_out_total_gr = df_out_total_gr.reset_index()
df_out_total_gr['Indicator'] = 'Total'

df_out_total = df_out_total.append(df_out_total_gr)

df_out_total_gr = df_out_total_gr.set_index(['Country Name'])
for count in df_tot.country.unique():
    df_country = df_tot[df_tot.country==count]
    a = np.where(df_country['value']<0)
    if len(a[0])>0:
        df_country = df_country[df_country.sector!='Land-Use Change and Forestry']
    ghg_tot_2018 = df_country.groupby('year').sum().loc[2018,'value']
    ghg_tot_2012 = df_country.groupby('year').sum().loc[2012,'value']
    df_out_total_gr.loc[count,'Value 2'] = (ghg_tot_2018 - ghg_tot_2012)/ghg_tot_2012
    

df_out_total_gr['Diff'] = df_out_total_gr['Value'] - df_out_total_gr['Value 2']
index_out = df_out_total_gr[np.abs(df_out_total_gr['Diff'])>0.001].index
df_out_total = df_out_total[~df_out_total['Country Name'].isin(index_out)]

data_out = pd.DataFrame()
for count in df_out_total['Country Name'].unique():
    data_temp = df_out_total[df_out_total['Country Name']==count]
    tot = data_temp.loc[data_temp.Indicator=='Total','Value'].values[0]
    data_temp.loc[data_temp.Indicator=='Total','Value'] = - 100
    data_temp = data_temp.sort_values('Value',ascending=False)
    data_temp['Ranking'] = range(1,len(data_temp)+1)
    data_temp.loc[data_temp.Indicator=='Total','Value'] = 0
    data_temp['Total Increase'] = tot
    data_out = data_out.append(data_temp)

data_out.to_excel('Data/Emissions/GHG_Growth_GDP_Pop.xlsx')

C:\Users\WB526885\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\WB526885\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\WB526885\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\WB526885\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\WB526885\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\WB526885\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in l

#### [ThinkHazard](https://gfdrr.github.io/thinkhazardmethods/#list-of-hazard-types-and-levels-for-a-division) + [list of countries](https://github.com/GFDRR/thinkhazardmethods/blob/master/source/download/ADM0_TH.csv)

In [85]:
list_countries = pd.read_excel('Data/ThinkHazard_List.xlsx')
df_full = pd.DataFrame()
for code,iso in zip(list_countries.ADM0_CODE,list_countries.ISO3166_a3):
    r = requests.get(' http://thinkhazard.org/en/report/%s.json'%code)
    x = r.json()
    df_country = pd.DataFrame([(x[i]['hazardtype']['hazardtype'],x[i]['hazardlevel']['title']) for i in range(len(x))],columns=['Hazard','Ranking'])
    df_country['Country Code'] = iso
    df_full = df_full.append(df_country)
    
df_full_t = df_full.copy()
df_full_t = df_full_t[df_full_t.Ranking!='No Data']
df_full_t = df_full_t[~df_full_t['Hazard'].isin(['Volcano','Water scarcity'])]
df_full_t = df_full_t.set_index(['Hazard','Ranking'])

dict_haz = {'EQ':'Earthquake','VO':'Vocalno','TS':'Tsunami','CY':'Cyclone','FL':'River flood', 'UF':'Urban flood', 
 'CF':'Coastal flood', 'LS':'Landslide','EH':'Extreme heat', 'WF':'Wildfire','DR':'Drought'}
dict_haz_flip = {v: k for k, v in dict_haz.items()}
dict_rank = {'LOW':'Low','VLO':'Very low', 'MED': 'Medium','HIG':'High'}
dict_rank_flip = {v: k for k, v in dict_rank.items()}

for haz,rank in df_full_t.index.unique():
    r = requests.get('https://thinkhazard.org/en/hazardcategory/%s/%s.json'%(dict_haz_flip[haz],dict_rank_flip[rank]))
    x = r.json()
    df_full_t.loc[(haz,rank),'Description'] = x['hazard_category']['general_recommendation'].split('.')[1]

df_full_t = df_full_t.reset_index().set_index('Country Code')
df_full_t['Country Name'] = iso_to_wb['country']
df_full_t = df_full_t.reset_index()
df_full_t.to_excel('Data/ThinkHazard.xlsx')


### [Global Climate Risk ](https://germanwatch.org/en/cri)

In [166]:
cri = pd.read_excel('Data/Global Climate Risk 99-18.xlsx')
cri = cri[cri.Country!='Serbia & Montenegro & Kosovo']
cri['ISO'] = cri['Country'].apply(lambda x: dict_map[x])
cri = cri.set_index('ISO')
cri['Country'] = iso_to_wb['country']
cri.reset_index().to_excel('Data/Climate Risk Index.xlsx')

#### World Road Statistics (WRS) Data - Ask Librarians to send data

In [22]:
wrs = pd.read_excel('Data/WRS_Country profiles.xlsx')
wrs = wrs.set_index('Country Code')
wrs['Country Name'] = iso_to_wb['country']
wrs = wrs.reset_index()

wrs.columns = [a + ' ' + str(b) for a,b in zip(wrs.columns,wrs.loc[0,:].values)]
wrs = wrs[1:]
wrs.to_excel('Data/WRS_Country_Profiles.xlsx')

### Floods

#### Flooding Costs without Adaptation - Future flood losses in major coastal cities Hallegatte et al. 2013

In [188]:
floods_cities = pd.read_excel('Data/Flooding_Costs_Cities.xlsx')
floods_cities['ISO'] = floods_cities.Country.apply(lambda x: dict_map[x])
floods_cities = floods_cities.set_index('ISO')
floods_cities['Country'] = iso_to_wb['country']
floods_cities.reset_index().to_excel('Data/Flooding_Costs_Cities_Hallegatte.xlsx')

#### Coastal Flood Protection - Beyond the Gap

In [198]:
cost_floods = pd.read_excel('Data/coastalflood_protection_country_level_results_over_GDP_preferred.xlsx')
cost_floods = cost_floods.set_index('Country Code')
cost_floods['Country Name'] = iso_to_wb['country']
cost_floods.reset_index().to_excel('Data/coastalflood_protection_levels.xlsx')

#### EMDAT

In [271]:
emdat = pd.read_csv('Data/EM-DAT.csv')
emdat = emdat[~emdat['Country Name'].isin(['Azores Islands','Saint Helena, Ascension and Tristan da Cunha','Cayman Islands (the)'
                            ,'Turks and Caicos Islands (the)','Saint Martin (French Part)','Sint Maarten (Dutch part)'
                            ,'Virgin Island (British)','Saint Barthélemy','Virgin Island (U.S.)','Canary Is',
                            'Northern Mariana Islands (the)','Cook Islands (the)'])]
emdat['ISO'] = emdat['Country Name'].apply(lambda x:dict_map[x])

emdat = emdat.set_index('ISO')
emdat['Country Name'] = iso_to_wb['country']
emdat.reset_index().to_excel('Data/EM-DAT.xlsx')

## US Energy Information Administration (EIA) - API 

### Total petroleum and other liquids (i.e. crude oil, NGPL, and other liquids - refinery processing gain) - Consumption

In [61]:
full_df = pd.DataFrame()
for country in countries.iso3:
    try:
        r = requests.get('http://api.eia.gov/series/?api_key=d640095c7a4e827101a6e192666d094b&series_id=INTL.5-2-%s-TBPD.A'%country)
        x = r.json()
        df = pd.DataFrame(x['series'][0]['data'])
        df.columns = ['Year','Value']
        df['Country Code'] = country
        full_df = full_df.append(df)
    except:
        pass

full_df = full_df.set_index('Country Code')
full_df['Country Name'] = iso_to_wb['country']
full_df = full_df.reset_index()

full_df = full_df[full_df.Value!='--']

pop = wb.download(indicator='SP.POP.TOTL',country='all',start=1980,end=2020)
pop = pop.reset_index().dropna()
pop.columns = ['Country Name','Year','Population']
pop = pop[~pop['Country Name'].isin(regions)]
pop = pop.set_index(['Country Name','Year'])

full_df = full_df.set_index(['Country Name','Year'])
full_df['Value_per_cap'] = full_df['Value'] / pop['Population']

full_df.reset_index().to_excel('Data/Fuel shift and efficiency in use sectors/Oil_Consumption.xlsx')

### Coke and coal production

In [80]:
full_df = pd.DataFrame()
for country in countries.iso3:
    try:
        r = requests.get('http://api.eia.gov/series/?api_key=d640095c7a4e827101a6e192666d094b&series_id=INTL.7-1-%s-TST.A'%country)
        x = r.json()
        df = pd.DataFrame(x['series'][0]['data'])
        df.columns = ['Year','Value']
        df['Country Code'] = country
        full_df = full_df.append(df)
    except:
        pass

full_df = full_df.set_index('Country Code')
full_df['Country Name'] = iso_to_wb['country']
full_df = full_df.reset_index()

full_df = full_df[full_df.Value!='--']

pop = wb.download(indicator='SP.POP.TOTL',country='all',start=1980,end=2020)
pop = pop.reset_index().dropna()
pop.columns = ['Country Name','Year','Population']
pop = pop[~pop['Country Name'].isin(regions)]
pop = pop.set_index(['Country Name','Year'])

full_df = full_df.set_index(['Country Name','Year'])
full_df['Value_per_cap'] = full_df['Value'] / pop['Population']

full_df.reset_index().to_excel('Data/Fuel shift and efficiency in use sectors/Coke_Coal_Consumption.xlsx')

### Dry Natural Gas Consumption

In [82]:
full_df = pd.DataFrame()
for country in countries.iso3:
    try:
        r = requests.get('http://api.eia.gov/series/?api_key=d640095c7a4e827101a6e192666d094b&series_id=INTL.26-2-%s-BCF.A'%country)
        x = r.json()
        df = pd.DataFrame(x['series'][0]['data'])
        df.columns = ['Year','Value']
        df['Country Code'] = country
        full_df = full_df.append(df)
    except:
        pass

full_df = full_df.set_index('Country Code')
full_df['Country Name'] = iso_to_wb['country']
full_df = full_df.reset_index()

full_df = full_df[full_df.Value!='--']

pop = wb.download(indicator='SP.POP.TOTL',country='all',start=1980,end=2020)
pop = pop.reset_index().dropna()
pop.columns = ['Country Name','Year','Population']
pop = pop[~pop['Country Name'].isin(regions)]
pop = pop.set_index(['Country Name','Year'])

full_df = full_df.set_index(['Country Name','Year'])
full_df['Value_per_cap'] = full_df['Value'] / pop['Population']

full_df.reset_index().to_excel('Data/Fuel shift and efficiency in use sectors/Gas_Consumption.xlsx')

## World Bank Data - API

#### CCKP

In [249]:
import time

In [250]:
df_out = pd.DataFrame()

for country in iso_to_wb.index:
    try:
        r = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v2/cru/country/pr/1991-2020/%s'%country)
        x = r.json()
        prec = pd.DataFrame(x)

        r = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v2/cru/country/tas/1991-2020/%s'%country)
        x = r.json()
        temp = pd.DataFrame(x)
        df_out = df_out.append(prec).append(temp)
    except:
        try:
            time.sleep(5)
            r = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v2/cru/country/pr/1991-2020/%s'%country)
            x = r.json()
            prec = pd.DataFrame(x)

            r = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v2/cru/country/tas/1991-2020/%s'%country)
            x = r.json()
            temp = pd.DataFrame(x)
            df_out = df_out.append(prec).append(temp)
        except:
            print(country)

df_out = df_out.set_index('Code')
df_out['Country Name'] = iso_to_wb['country']
df_out = df_out.reset_index()
df_out[df_out.variable=='pr'].to_excel('Data/CCKP_hist_precipitation.xlsx')
df_out[df_out.variable=='tas'].to_excel('Data/CCKP_hist_temperature.xlsx')

#### Time Series Temperature

In [33]:
temp_proj_full = pd.DataFrame()
for country in iso_to_wb.index:
    
    url = 'http://climatedataapi.worldbank.org/datamanager/getTimeSeriesData'
    body = {"variable":"tas","model":"all","rcp":None,"statistics":["stddevlow","median","stddevhigh"],"period":"1986-2099",
            "spatialId":"country","spaceId":country}

    r = requests.post(url,json=body)
    k = r.json()
    k = pd.DataFrame(k['timeSeries'])
    
    for i in range(len(k)):
        temp_proj = pd.DataFrame()
        temp_proj['Year'] = range(1986,2100)
        temp_proj['temp'] = k.loc[i,'data']
        temp_proj['Model'] = k.loc[i,'model'].split('-')[2]
        temp_proj['Statistic'] = k.loc[i,'statistic']
        temp_proj['Country Code'] = country
        temp_proj_full = temp_proj_full.append(temp_proj)

temp_proj_full = temp_proj_full.dropna(subset=['temp'])
temp_proj_full = temp_proj_full.set_index('Country Code')
temp_proj_full['Country Name'] = iso_to_wb['country']

temp_proj_full_1 = temp_proj_full[temp_proj_full.Model.isin(['historical','rcp26'])]
temp_proj_full_1['Model'] = 'RCP26'

temp_proj_full_2 = temp_proj_full[temp_proj_full.Model.isin(['historical','rcp85'])]
temp_proj_full_2['Model'] = 'RCP85'

temp_proj_full_fin = temp_proj_full_1.append(temp_proj_full_2)
temp_proj_full_fin = temp_proj_full_fin.reset_index()

temp_proj_full_fin_med = temp_proj_full_fin[temp_proj_full_fin.Statistic=='median']
temp_proj_full_fin_stdH = temp_proj_full_fin[temp_proj_full_fin.Statistic=='stddevhigh']
temp_proj_full_fin_stdL = temp_proj_full_fin[temp_proj_full_fin.Statistic=='stddevlow']

temp_proj_full_fin_med = temp_proj_full_fin_med.set_index(['Country Name','Country Code','Year','Model'])
temp_proj_full_fin_stdH = temp_proj_full_fin_stdH.set_index(['Country Name','Country Code','Year','Model'])
temp_proj_full_fin_stdL = temp_proj_full_fin_stdL.set_index(['Country Name','Country Code','Year','Model'])

temp_proj_full_fin_med['Temp_High'] = temp_proj_full_fin_stdH['temp']
temp_proj_full_fin_med['Temp_Low'] = temp_proj_full_fin_stdL['temp']

temp_proj_full_fin_med.reset_index().to_excel('Data/Temperature_Projections.xlsx')

<ipython-input-33-b9b37297a445>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_proj_full_1['Model'] = 'RCP26'
<ipython-input-33-b9b37297a445>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_proj_full_2['Model'] = 'RCP85'


#### GDP

In [202]:
gdp = wb.download(indicator='NY.GDP.MKTP.CD',country='all',start=1990,end=2020)
gdp = gdp.reset_index().dropna()
gdp.columns = ['Country Name','Year','GDP']
gdp = gdp[~gdp['Country Name'].isin(regions)]

gdp.to_excel('Data/Social Profile/GDP.xlsx')

#### GDP Growth and [GHG Growth](https://www.climatewatchdata.org/ghg-emissions?calculation=PERCENTAGE_CHANGE&end_year=2018&source=CAIT&start_year=1990)

In [422]:
gdp_gr = wb.download(indicator='NY.GDP.MKTP.KD.ZG',country='all',start=1990,end=2020)
gdp_gr = gdp_gr.reset_index().dropna()
gdp_gr.columns = ['Country Name','Year','GDP_Growth']
gdp_gr = gdp_gr[~gdp_gr['Country Name'].isin(regions)]

gdp_gr.columns = ['Country Name','Year','Value']
gdp_gr['Indicator'] = 'GDP'

ghg_growth = pd.read_csv('Data/Emissions/ghg-emissions-growth-CAIT.csv')
ghg_growth = ghg_growth[:-2].drop(['unit','1990'],axis=1)
ghg_growth['Country Code'] = ghg_growth['Country/Region'].apply(lambda x : dict_map[x])
ghg_growth = ghg_growth.set_index('Country Code')
ghg_growth['Country Name'] = iso_to_wb['country']
ghg_growth = ghg_growth.reset_index()
ghg_growth = ghg_growth.drop(['Country/Region'],axis=1)
ghg_growth = pd.DataFrame(ghg_growth.set_index(['Country Name','Country Code']).stack()).reset_index()

ghg_growth.columns = ['Country Name','Country Code','Year','Value']
ghg_growth['Indicator'] = 'GHG'

ghg_growth.append(gdp_gr).drop(['Country Code'],axis=1).to_excel('Data/Emissions/GHG_GDP_decoupling.xlsx')

#### GDP per capita

In [203]:
gdp = wb.download(indicator='NY.GDP.PCAP.CD',country='all',start=1990,end=2020)
gdp = gdp.reset_index().dropna()
gdp.columns = ['Country Name','Year','GDP']
gdp = gdp[~gdp['Country Name'].isin(regions)]

gdp.to_excel('Data/Social Profile/GDP_per_capita.xlsx')

#### Population

In [7]:
pop = wb.download(indicator='SP.POP.TOTL',country='all',start=1990,end=2020)
pop = pop.reset_index().dropna()
pop.columns = ['Country Name','Year','Population']
pop = pop[~pop['Country Name'].isin(regions)]

pop.to_excel('Data/Social Profile/population.xlsx')

#### Share urban population 

In [157]:
share_urban = wb.download(indicator='SP.URB.TOTL.IN.ZS',country='all',start=1980,end=2020)
share_urban = share_urban.reset_index().dropna()
share_urban.columns = ['Country Name','Year','Share urban population (%)']
# share_urban = get_latest_year(share_urban)
share_urban.to_excel('Data/Social Profile/Share_urban_population.xlsx')

#### Urbanization growth

In [9]:
urb_growth = wb.download(indicator='SP.URB.GROW',country='all',start=1980,end=2020)
urb_growth = urb_growth.reset_index().dropna()
urb_growth.columns = ['Country Name','Year','Urbanization growth (%)']
urb_growth = get_latest_year(urb_growth)
urb_growth.to_excel('Data/Social Profile/Urbanization_growth.xlsx')

#### Renewable internal freshwater resources per capita

In [ ]:
water = wb.download(indicator='ER.H2O.INTR.PC',country='all',start=1980,end=2020)
water = water.reset_index().dropna()
water.columns = ['Country Name','Year','Ren water resources cap']
water = get_latest_year(water)
water.to_excel('Data/Social Profile/Renewable_Water_Resources_cap.xlsx')

#### Gini

In [87]:
gini = wb.download(indicator='SI.POV.GINI',country='all',start=1980,end=2020)
gini = gini.reset_index().dropna()
gini.columns = ['Country Name','Year','Gini']
gini = get_latest_year(gini)
gini.to_excel('Data/Social Profile/Gini.xlsx')

#### Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population)

In [88]:
pov = wb.download(indicator='SI.POV.UMIC',country='all',start=1980,end=2020)
pov = pov.reset_index().dropna()
pov.columns = ['Country Name','Year','Poverty Headcount Ratio 5.5 (2011 PPP)']
pov = get_latest_year(pov)
pov.to_excel('Data/Social Profile/Poverty_Headcount_Ratio_55.xlsx')

#### [Education attainment by level](https://datacatalog.worldbank.org/dataset/global-jobs-indicators-database?deliveryName=DM95330)

In [129]:
edu_attainment = pd.read_excel('Data/Social Profile/join_database.xlsx',skiprows=3)

edu_attainment = edu_attainment[['Countryname', 'Country code', 'Region', 'Subsample',
       'Income Level Name', 'Income Level Code', 'Year of survey',' No Education', ' Primary Education', ' Secondary Education',
       ' Post Secondary Education','Mean number of years of education completed, aged 17 and older']]
edu_attainment = edu_attainment.rename({'Countryname':'Country Name'},axis=1)
edu_attainment = edu_attainment.rename({'Year of survey':'Year'},axis=1)
edu_attainment = get_latest_year(edu_attainment)

edu_attainment = pd.DataFrame(edu_attainment.set_index(['Country Name','Country code','Region', 'Subsample',
       'Income Level Name', 'Income Level Code', 'Year']).stack()).reset_index()
edu_attainment.columns = ['Country Name','Country code','Region', 'Subsample',
       'Income Level Name', 'Income Level Code', 'Year','Indicator','Value']

edu_attainment = edu_attainment.set_index('Country code')
edu_attainment['Country Name'] = iso_to_wb['country']

edu_attainment.reset_index().to_excel('Data/Social Profile/Education_attainment.xlsx')

#### Employment by sector

In [128]:
emp_sec = pd.read_excel('Data/Social Profile/join_database.xlsx',skiprows=3)
emp_sec = emp_sec[['Countryname', 'Country code', 'Region', 'Subsample',
       'Income Level Name', 'Income Level Code', 'Year of survey','Public sector employment, aged 15-64',
       ' Agriculture, aged 15-64', ' Industry, aged 15-64',' Services, aged 15-64', 'Mining, aged 15-64',
       'Manufacturing, aged 15-64','Electricity and utilities, aged 15-64','Construction, aged 15-64', 'Commerce, aged 15-64',
       'Transport & Communication, aged 15-64','Financial and Business Services, aged 15-64',
       'Public Administration, aged 15-64', 'Other services, aged 15-64']]

emp_sec = emp_sec.rename({'Countryname':'Country Name'},axis=1)
emp_sec = emp_sec.rename({'Year of survey':'Year'},axis=1)
emp_sec = pd.DataFrame(emp_sec.set_index(['Country Name', 'Country code', 'Region', 'Subsample',
       'Income Level Name', 'Income Level Code', 'Year']).stack()).reset_index()
emp_sec.columns = ['Country Name', 'Country code', 'Region', 'Subsample',
       'Income Level Name', 'Income Level Code', 'Year','Indicator','Value']

emp_sec = get_latest_year(emp_sec)
emp_sec = emp_sec.set_index('Country code')
emp_sec['Country Name'] = iso_to_wb['country']

emp_sec.reset_index().to_excel('Data/Social Profile/Employment_sector.xlsx')

#### Literacy rate

In [39]:
literacy_rate = wb.download(indicator='SE.ADT.LITR.ZS',country='all',start=1990,end=2020)
literacy_rate = literacy_rate.reset_index().dropna()
literacy_rate.columns = ['Country Name','Year','Literacy Rate (% of population)']
literacy_rate = get_latest_year(literacy_rate)
literacy_rate = literacy_rate[~literacy_rate['Country Name'].isin(regions)]

literacy_rate.to_excel('Data/Social Profile/literacy_rate.xlsx')

#### Access to electricity

In [650]:
elec = wb.download(indicator='EG.ELC.ACCS.ZS',country='all',start=1990,end=2020)
elec = elec.reset_index().dropna()
elec.columns = ['Country Name','Year','Access to electricity(% tot pop)']
elec = get_latest_year(elec)
elec = elec[~elec['Country Name'].isin(regions)]

elec.to_excel('Data/Decarbonization of the power sector/access_elec.xlsx')

#### Share of firms experiencing eletrical outages

In [664]:
firms_pow = wb.download(indicator='IC.ELC.OUTG.ZS',country='all',start=2000,end=2020)
firms_pow = firms_pow.reset_index().dropna()
firms_pow.columns = ['Country Name','Year','% Share of firms experiencing power outages']
firms_pow = get_latest_year(firms_pow)
firms_pow = firms_pow.sort_values('% Share of firms experiencing power outages')
firms_pow = firms_pow[~firms_pow['Country Name'].isin(regions)]

firms_pow['Ranking'] = range(1,len(firms_pow)+1)
firms_pow.to_excel('Data/Decarbonization of the power sector/%firms_elec_outages.xlsx')

#### Share of sales lost for firms subject to power outages

In [665]:
pow_lost = wb.download(indicator='IC.FRM.OUTG.ZS',country='all',start=2000,end=2020)
pow_lost = pow_lost.reset_index().dropna()
pow_lost.columns = ['Country Name','Year','% Sales lost to power outages']
pow_lost = get_latest_year(pow_lost)
pow_lost = pow_lost.sort_values('% Sales lost to power outages')
pow_lost = pow_lost[~pow_lost['Country Name'].isin(regions)]
pow_lost['Ranking'] = range(1,len(pow_lost)+1)

pow_lost.to_excel('Data/Decarbonization of the power sector/%sales_lost_power.xlsx')

#### Power outages (Power outages in firms in a typical month)

In [666]:
nb_pow_month = wb.download(indicator='IC.ELC.OUTG',country='all',start=2000,end=2020)
nb_pow_month = nb_pow_month.reset_index().dropna()
nb_pow_month.columns = ['Country Name','Year','Number power outages typical month']
nb_pow_month = get_latest_year(nb_pow_month)

nb_pow_month = nb_pow_month.sort_values('Number power outages typical month')
nb_pow_month = nb_pow_month[~nb_pow_month['Country Name'].isin(regions)]
nb_pow_month['Ranking'] = range(1,len(nb_pow_month)+1)

nb_pow_month.to_excel('Data/Decarbonization of the power sector/nb_power_out.xlsx')

#### Mileage per ton of freight transport, air transport

In [425]:
air_transp_freight = wb.download(indicator='IS.AIR.GOOD.MT.K1',country='all',start=2000,end=2020)
air_transp_freight = air_transp_freight.reset_index().dropna()
air_transp_freight.columns = ['Country Name','Year','Freight - Air Transport ton-km']
air_transp_freight = get_latest_year(air_transp_freight)
air_transp_freight.to_excel('Data/Fuel shift and efficiency in use sectors/freight_air_transport.xlsx')

#### People using at least basic sanitation services

In [801]:
sani_services = wb.download(indicator='SH.STA.BASS.ZS',country='all',start=2000,end=2020)
sani_services = sani_services.reset_index().dropna()
sani_services.columns = ['Country Name','Year','People using sanit services (%)']
sani_services = sani_services[~sani_services['Country Name'].isin(regions)]
sani_services = get_latest_year(sani_services)
sani_services = sani_services.sort_values('People using sanit services (%)',ascending=False)

i = 1
sani_services.loc[sani_services['People using sanit services (%)']>=100,'People using sanit services (%)'] = 100
for val in sani_services['People using sanit services (%)'].unique():
    sani_services.loc[sani_services['People using sanit services (%)']==val,'Ranking'] = i
    i = i + 1

sani_services.to_excel('Data/Fuel shift and efficiency in use sectors/people_using_basic_sanitation.xlsx')

#### People using at least basic drinking water services (%of population)

In [808]:
drink_water = wb.download(indicator='SH.H2O.BASW.ZS',country='all',start=2000,end=2020)
drink_water = drink_water.reset_index().dropna()
drink_water.columns = ['Country Name','Year','People using basic drinking water services (%)']
drink_water = drink_water[~drink_water['Country Name'].isin(regions)]
drink_water = get_latest_year(drink_water)

drink_water = drink_water.sort_values('People using basic drinking water services (%)',ascending=False)

i = 1
drink_water.loc[drink_water['People using basic drinking water services (%)']>=100,'People using basic drinking water services (%)'] = 100
for val in drink_water['People using basic drinking water services (%)'].unique():
    drink_water.loc[drink_water['People using basic drinking water services (%)']==val,'Ranking'] = i
    i = i + 1


drink_water.to_excel('Data/Fuel shift and efficiency in use sectors/people_using_basic_drinking.xlsx')

#### People with basic handwashing facilities including soap and water

In [812]:
handwash_fac = wb.download(indicator='SH.STA.HYGN.ZS',country='all',start=2000,end=2020)
handwash_fac = handwash_fac.reset_index().dropna()
handwash_fac.columns = ['Country Name','Year','People using basic handwashing (%)']
handwash_fac = handwash_fac[~handwash_fac['Country Name'].isin(regions)]
handwash_fac = get_latest_year(handwash_fac)

handwash_fac = handwash_fac.sort_values('People using basic handwashing (%)',ascending=False)

i = 1
handwash_fac.loc[handwash_fac['People using basic handwashing (%)']>=100,'People using basic handwashing (%)'] = 100
for val in handwash_fac['People using basic handwashing (%)'].unique():
    handwash_fac.loc[handwash_fac['People using basic handwashing (%)']==val,'Ranking'] = i
    i = i + 1

handwash_fac.to_excel('Data/Fuel shift and efficiency in use sectors/people_using_basic_handwashing.xlsx')

#### Natural gas rents

In [873]:
nat_gas_rents = wb.download(indicator='NY.GDP.NGAS.RT.ZS',country='all',start=2000,end=2020)
nat_gas_rents = nat_gas_rents.reset_index().dropna()
nat_gas_rents.columns = ['Country Name','Year','Value']
nat_gas_rents['Indicator'] = 'Natural gas rents (% of GDP)'
nat_gas_rents = get_latest_year(nat_gas_rents)
# nat_gas_rents.to_excel('Data/Fuel shift and efficiency in use sectors/gas_rents.xlsx')

#### Oil rents

In [874]:
oil_rents = wb.download(indicator='NY.GDP.PETR.RT.ZS',country='all',start=2000,end=2020)
oil_rents = oil_rents.reset_index().dropna()
oil_rents.columns = ['Country Name','Year','Value']
oil_rents = get_latest_year(oil_rents)
oil_rents['Indicator'] = 'Oil rents (% of GDP)'
# oil_rents.to_excel('Data/Fuel shift and efficiency in use sectors/oil_rents.xlsx')

#### Coal rents

In [875]:
coal_rents = wb.download(indicator='NY.GDP.COAL.RT.ZS',country='all',start=2000,end=2020)
coal_rents = coal_rents.reset_index().dropna()
coal_rents.columns = ['Country Name','Year','Value']
coal_rents = get_latest_year(coal_rents)
coal_rents['Indicator'] = 'Coal rents (% of GDP)'
# coal_rents.to_excel('Data/Fuel shift and efficiency in use sectors/coal_rents.xlsx')

#### Mineral rents

In [876]:
min_rents = wb.download(indicator='NY.GDP.MINR.RT.ZS',country='all',start=2000,end=2020)
min_rents = min_rents.reset_index().dropna()
min_rents.columns = ['Country Name','Year','Value']
min_rents = get_latest_year(min_rents)
min_rents['Indicator'] = 'Mineral rents (% of GDP)'
# min_rents.to_excel('Data/Fuel shift and efficiency in use sectors/mineral_rents.xlsx')

#### Total natural resources rents

In [877]:
tot_nat_rents = wb.download(indicator='NY.GDP.TOTL.RT.ZS',country='all',start=2000,end=2020)
tot_nat_rents = tot_nat_rents.reset_index().dropna()
tot_nat_rents.columns = ['Country Name','Year','Value']
tot_nat_rents['Indicator'] = 'Total natural resources rents (% of GDP)'
tot_nat_rents = get_latest_year(tot_nat_rents)

tot_rents = tot_nat_rents.append(min_rents).append(coal_rents).append(oil_rents).append(nat_gas_rents)
tot_rents = tot_rents[~tot_rents['Country Name'].isin(regions)]
tot_rents.to_excel('Data/Fuel shift and efficiency in use sectors/multiple_rents.xlsx')

# tot_nat_rents.to_excel('Data/Fuel shift and efficiency in use sectors/total_natural_res_rents.xlsx')

#### Share Arable land

In [396]:
share_ar_land = wb.download(indicator='AG.LND.ARBL.ZS',country='all',start=2000,end=2020)
share_ar_land = share_ar_land.reset_index().dropna()
share_ar_land.columns = ['Country Name','Year','Share of arable land (%)']
share_ar_land = get_latest_year(share_ar_land)
share_ar_land.to_excel('Data/Food, land-use and natural sink/share_arable_land.xlsx')

#### Share Ag land

In [458]:
share_ag_land = wb.download(indicator='AG.LND.AGRI.ZS',country='all',start=2000,end=2020)
share_ag_land = share_ag_land.reset_index().dropna()
share_ag_land.columns = ['Country Name','Year','Share of ag land (%)']
share_ag_land = get_latest_year(share_ag_land)
share_ag_land.to_excel('Data/Food, land-use and natural sink/share_agricultural_land.xlsx')

#### Labor force participation rate

In [491]:
labor_force_particip_rate = wb.download(indicator='SL.TLF.ACTI.ZS',country='all',start=2000,end=2020)
labor_force_particip_rate = labor_force_particip_rate.reset_index().dropna()
labor_force_particip_rate.columns = ['Country Name','Year','Labor force participation rate (%)']
labor_force_particip_rate = get_latest_year(labor_force_particip_rate)
labor_force_particip_rate.to_excel('Data/Managing the transition/Labor_force_participation_rate.xlsx')

#### Female labor force participation rate

In [492]:
fem_labor_force_particip_rate = wb.download(indicator='SL.TLF.CACT.FE.ZS',country='all',start=2000,end=2020)
fem_labor_force_particip_rate = fem_labor_force_particip_rate.reset_index().dropna()
fem_labor_force_particip_rate.columns = ['Country Name','Year','Female labor force participation rate (%)']
fem_labor_force_particip_rate = get_latest_year(fem_labor_force_particip_rate)
fem_labor_force_particip_rate.to_excel('Data/Managing the transition/Femal_labor_part_rate.xlsx')

#### Debt level

In [505]:
debt_level = wb.download(indicator='GC.DOD.TOTL.GD.ZS',country='all',start=2000,end=2020)
debt_level = debt_level.reset_index().dropna()
debt_level.columns = ['Country Name','Year','Debt level (% of GDP)']
debt_level = get_latest_year(debt_level)
debt_level.to_excel('Data/Managing the transition/Debt_level.xlsx')

#### Total tax revenue

In [508]:
total_tax_rev = wb.download(indicator='GC.TAX.TOTL.GD.ZS',country='all',start=2000,end=2020)
total_tax_rev = total_tax_rev.reset_index().dropna()
total_tax_rev.columns = ['Country Name','Year','Total tax revenue (%)']
total_tax_rev = get_latest_year(total_tax_rev)
total_tax_rev.to_excel('Data/Managing the transition/total_tax_revenue.xlsx')

#### Coverage of social protection (all social assistance)

In [489]:
cov = wb.download(indicator='per_sa_allsa.cov_pop_tot',country='all',start=2000,end=2020)
cov = cov.reset_index().dropna()
cov.columns = ['Country Name','Year','Coverage social assistance (% of pop)']
cov = get_latest_year(cov)
cov.to_excel('Data/Managing the transition/coverage_social_protection.xlsx')

#### Unemployment

In [490]:
unemp = wb.download(indicator='SL.UEM.TOTL.ZS',country='all',start=2000,end=2020)
unemp = unemp.reset_index().dropna()
unemp.columns = ['Country Name','Year','Unemployment (% of population)']
unemp = get_latest_year(unemp)
unemp.to_excel('Data/Managing the transition/unemployment.xlsx')

#### School Enrollment Primary Male

In [494]:
primary_male = wb.download(indicator='SE.PRM.ENRR.MA',country='all',start=2000,end=2020)
primary_male = primary_male.reset_index().dropna()
primary_male.columns = ['Country Name','Year','School enrollment, primary, male (% gross)']
primary_male = get_latest_year(primary_male)

#### School Enrollment Primary Female

In [495]:
primary_female = wb.download(indicator='SE.PRM.ENRR.FE',country='all',start=2000,end=2020)
primary_female = primary_female.reset_index().dropna()
primary_female.columns = ['Country Name','Year','School enrollment, primary, female (% gross)']
primary_female = get_latest_year(primary_female)

#### School Enrollment Secondary Female

In [496]:
sec_female = wb.download(indicator='SE.SEC.ENRR.FE',country='all',start=2000,end=2020)
sec_female = sec_female.reset_index().dropna()
sec_female.columns = ['Country Name','Year','School enrollment, secondary, female (% gross)']
sec_female = get_latest_year(sec_female)

#### School Enrollment Secondary Male

In [497]:
sec_male = wb.download(indicator='SE.SEC.ENRR.MA',country='all',start=2000,end=2020)
sec_male = sec_male.reset_index().dropna()
sec_male.columns = ['Country Name','Year','School enrollment, secondary, male (% gross)']
sec_male = get_latest_year(sec_male)

#### School Enrollment Tertiary Female

In [498]:
ter_female = wb.download(indicator='SE.TER.ENRR.FE',country='all',start=2000,end=2020)
ter_female = ter_female.reset_index().dropna()
ter_female.columns = ['Country Name','Year','School enrollment, tertiary, female (% gross)']
ter_female = get_latest_year(ter_female)

#### School Enrollment Tertiary Male

In [499]:
ter_male = wb.download(indicator='SE.TER.ENRR.MA',country='all',start=2000,end=2020)
ter_male = ter_male.reset_index().dropna()
ter_male.columns = ['Country Name','Year','School enrollment, tertiary, male (% gross)']
ter_male = get_latest_year(ter_male)

primary_male['Type'] = 'Primary Male'
primary_male.columns = ['Country Name','Year','Value','Type']

primary_female['Type'] = 'Primary Female'
primary_female.columns = ['Country Name','Year','Value','Type']

sec_male['Type'] = 'Secondary Male'
sec_male.columns = ['Country Name','Year','Value','Type']

sec_female['Type'] = 'Secondary Female'
sec_female.columns = ['Country Name','Year','Value','Type']

ter_female['Type'] = 'Tertiary female'
ter_female.columns = ['Country Name','Year','Value','Type']

ter_male['Type'] = 'Tertiary Male'
ter_male.columns = ['Country Name','Year','Value','Type']

school_enroll = primary_male.append(primary_female).append(sec_male).append(sec_female).append(ter_female).append(ter_male)
school_enroll.to_excel('Data/Managing the transition/School_enrollement.xlsx')

#### Fixed broadband subscriptions (per 100 people)

In [493]:
internet = wb.download(indicator='IT.NET.BBND.P2',country='all',start=2000,end=2020)
internet = internet.reset_index().dropna()
internet.columns = ['Country Name','Year','Fixed broadband subscriptions (per 100 people)']
internet = get_latest_year(internet)
internet.to_excel('Data/Managing the transition/Fixed broadband subscriptions.xlsx')

#### Social Protection and Labor - Poverty Headcount Reduction

In [39]:
pov_red = wb.download(indicator='per_allsp_p0_preT_tot',country='all',start=2000,end=2020)
pov_red = pov_red.reset_index().dropna()
pov_red.columns = ['Country Name','Year','Poverty Headcount Reduction (%)']
pov_red = get_latest_year(pov_red)
pov_red = pov_red[~pov_red['Country Name'].isin(regions)]

pov_red.to_excel('Data/Residual Risk Management/ASP_Poverty_Red.xlsx')

#### Social Protection and Labor - Average Transfert Amount

In [41]:
av_tran = wb.download(indicator='per_allsp.avt_pop_preT_tot',country='all',start=2000,end=2020)
av_tran = av_tran.reset_index().dropna()
av_tran.columns = ['Country Name','Year','Average per capita transfer ($ PPP/day)']
av_tran = get_latest_year(av_tran)
av_tran = av_tran[~av_tran['Country Name'].isin(regions)]

av_tran.to_excel('Data/Residual Risk Management/ASP_Average_per_capita_transfer.xlsx')

#### Social Protection and Labor - Gini Reduction

In [43]:
gini_red = wb.download(indicator='per_allsp_gini_preT_tot',country='all',start=2000,end=2020)
gini_red = gini_red.reset_index().dropna()
gini_red.columns = ['Country Name','Year','Gini Reduction (%)']
gini_red = get_latest_year(gini_red)
gini_red = gini_red[~gini_red['Country Name'].isin(regions)]

gini_red.to_excel('Data/Residual Risk Management/ASP_Gini_Red.xlsx')

#### Social Protection and Labor - Coverage 1st quintile

In [44]:
coverage_1st = wb.download(indicator='per_allsp.cov_q1_tot',country='all',start=2000,end=2020)
coverage_1st = coverage_1st.reset_index().dropna()
coverage_1st.columns = ['Country Name','Year','Coverage (1st quintile)']
coverage_1st = get_latest_year(coverage_1st)
coverage_1st = coverage_1st[~coverage_1st['Country Name'].isin(regions)]

coverage_1st.to_excel('Data/Residual Risk Management/ASP_Coverage_1st.xlsx')

#### Social Protection and Labor

In [167]:
coverage_tot = wb.download(indicator='per_allsp.cov_pop_tot',country='all',start=2000,end=2020)
coverage_tot = coverage_tot.reset_index().dropna()
coverage_tot.columns = ['Country Name','Year','Coverage']
coverage_tot = get_latest_year(coverage_tot)
coverage_tot = coverage_tot[~coverage_tot['Country Name'].isin(regions)]

coverage_tot.to_excel('Data/Residual Risk Management/ASP_Coverage_tot.xlsx')

#### Financial Inclusion - Account

In [51]:
account = wb.download(indicator='account.t.d',country='all',start=2000,end=2020)
account = account.reset_index().dropna()
account.columns = ['Country Name','Year','Account']
account = get_latest_year(account)
account = account[~account['Country Name'].isin(regions)]

account.to_excel('Data/Residual Risk Management/Findex_Account.xlsx')

#### Financial Inclusion - Account, Income 40%

In [53]:
account = wb.download(indicator='account.t.d.7',country='all',start=2000,end=2020)
account = account.reset_index().dropna()
account.columns = ['Country Name','Year','Account,income, poorest 40%']
account = get_latest_year(account)
account = account[~account['Country Name'].isin(regions)]

account.to_excel('Data/Residual Risk Management/Findex_Account_40%.xlsx')

#### Financial Inclusion - Digital Payments

In [50]:
dig_pay = wb.download(indicator='g20.t.made',country='all',start=2000,end=2020)
dig_pay = dig_pay.reset_index().dropna()
dig_pay.columns = ['Country Name','Year','Digital payments last year(%)']
dig_pay = get_latest_year(dig_pay)
dig_pay = dig_pay[~dig_pay['Country Name'].isin(regions)]

dig_pay.to_excel('Data/Residual Risk Management/Findex_Digital_Payments.xlsx')

#### Financial Inclusion - Saving money 

In [55]:
savings = wb.download(indicator='fin18.t.d',country='all',start=2000,end=2020)
savings = savings.reset_index().dropna()
savings.columns = ['Country Name','Year','Saving money in the past year']
savings = get_latest_year(savings)
savings = savings[~savings['Country Name'].isin(regions)]

savings.to_excel('Data/Residual Risk Management/Findex_Saving_Money.xlsx')

#### Governance Effectiveness

In [22]:
gov_eff = wb.download(indicator='GE.EST',country='all',start=2000,end=2020)
gov_eff = gov_eff.reset_index().dropna()
gov_eff.columns = ['Country Name','Year','Governance Effectiveness']
gov_eff = get_latest_year(gov_eff)
gov_eff = gov_eff[~gov_eff['Country Name'].isin(regions)]
gov_eff = gov_eff.sort_values('Governance Effectiveness',ascending=False)
gov_eff['Ranking'] = range(1,len(gov_eff)+1)
gov_eff.to_excel('Data/Managing the transition/Gov_Effec.xlsx')

#### Political stability and absence of violence/terrorism

In [23]:
poli_sta = wb.download(indicator='PV.EST',country='all',start=2000,end=2020)
poli_sta = poli_sta.reset_index().dropna()
poli_sta.columns = ['Country Name','Year','Political stability and absence of violence']
poli_sta = get_latest_year(poli_sta)
poli_sta = poli_sta[~poli_sta['Country Name'].isin(regions)]
poli_sta = poli_sta.sort_values('Political stability and absence of violence',ascending=False)
poli_sta['Ranking'] = range(1,len(poli_sta)+1)
poli_sta.to_excel('Data/Managing the transition/Political_Stability.xlsx')

#### Regulatory quality

In [24]:
regu_qua = wb.download(indicator='RQ.EST',country='all',start=2000,end=2020)
regu_qua = regu_qua.reset_index().dropna()
regu_qua.columns = ['Country Name','Year','Regulatory Quality']
regu_qua = get_latest_year(regu_qua)
regu_qua = regu_qua[~regu_qua['Country Name'].isin(regions)]
regu_qua = regu_qua.sort_values('Regulatory Quality',ascending=False)
regu_qua['Ranking'] = range(1,len(regu_qua)+1)
regu_qua.to_excel('Data/Managing the transition/Regulatory_Quality.xlsx')

#### Rule of law

In [29]:
rule_law = wb.download(indicator='RL.EST',country='all',start=2000,end=2020)
rule_law = rule_law.reset_index().dropna()
rule_law.columns = ['Country Name','Year','Rule of law']
rule_law = get_latest_year(rule_law)
rule_law = rule_law[~rule_law['Country Name'].isin(regions)]
rule_law = rule_law.sort_values('Rule of law',ascending=False)
rule_law['Ranking'] = range(1,len(rule_law)+1)
rule_law.to_excel('Data/Managing the transition/Rule_of_law.xlsx')

#### Voice and accountability

In [33]:
voice = wb.download(indicator='VA.EST',country='all',start=2000,end=2020)
voice = voice.reset_index().dropna()
voice.columns = ['Country Name','Year','Voice and accountability']
voice = get_latest_year(voice)
voice = voice[~voice['Country Name'].isin(regions)]
voice = voice.sort_values('Voice and accountability',ascending=False)
voice['Ranking'] = range(1,len(voice)+1)
voice.to_excel('Data/Managing the transition/Voice_accountability.xlsx')

## Non-API

#### Financial Inclusion - Account, Income 40%

In [47]:
account = wb.download(indicator='per_allsp_p0_preT_tot',country='all',start=2000,end=2020)
account = account.reset_index().dropna()
account.columns = ['Country Name','Year','Account,income, poorest 40%']
account = get_latest_year(account)
account = account[~account['Country Name'].isin(regions)]

account.to_excel('Data/Residual Risk Management/Findex_Account_40%.xlsx')

#### [GHG per GDP](https://www.climatewatchdata.org/ghg-emissions?calculation=PER_GDP&end_year=2018&start_year=1990) CAIT

In [439]:
ghg_per_GDP = pd.read_csv('Data/Emissions/ghg-emissions-GDP-CAIT.csv')
ghg_per_GDP = ghg_per_GDP[:-2]
ghg_per_GDP['Country Code'] = ghg_per_GDP['Country/Region'].apply(lambda x : dict_map[x])
ghg_per_GDP = ghg_per_GDP.set_index('Country Code')
ghg_per_GDP['Country Name'] = iso_to_wb['country']
ghg_per_GDP = ghg_per_GDP.reset_index()
ghg_per_GDP = ghg_per_GDP.drop(['Country/Region'],axis=1)
ghg_per_GDP = pd.DataFrame(ghg_per_GDP.set_index(['Country Name','Country Code','unit']).stack()).reset_index()
ghg_per_GDP.columns = ['Country Name','Country Code','unit','Year','Value']
ghg_per_GDP = ghg_per_GDP[ghg_per_GDP.Value!='false']
ghg_per_GDP.to_excel('Data/Emissions/GHG_per_GDP.xlsx')


#### [GHG per capita](https://www.climatewatchdata.org/ghg-emissions?calculation=PER_CAPITA&end_year=2018&start_year=1990) CAIT

In [445]:
ghg_per_capita = pd.read_csv('Data/Emissions/ghg-emissions-capita-CAIT.csv')
ghg_per_capita = ghg_per_capita[:-2]
ghg_per_capita['Country Code'] = ghg_per_capita['Country/Region'].apply(lambda x : dict_map[x])
ghg_per_capita = ghg_per_capita.set_index('Country Code')
ghg_per_capita['Country Name'] = iso_to_wb['country']
ghg_per_capita = ghg_per_capita.reset_index()
ghg_per_capita = ghg_per_capita.drop(['Country/Region'],axis=1)
ghg_per_capita = pd.DataFrame(ghg_per_capita.set_index(['Country Name','Country Code','unit']).stack()).reset_index()
ghg_per_capita.columns = ['Country Name','Country Code','unit','Year','Value']
ghg_per_capita = ghg_per_capita[ghg_per_capita.Value!='false']
ghg_per_capita.to_excel('Data/Emissions/GHG_per_capita.xlsx')


#### [Youth and adults able to write a computer program using a specialized programming language](https://unstats-undesa.opendata.arcgis.com/datasets/c86e872cc853477180d39477bcde9f88) via https://country-profiles.unstatshub.org/pak#goal-4

In [139]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/SE_ADT_ACTS_4_4_1_2020Q2G03/FeatureServer/0/query?where=typeOfSkill_desc%20%3D%20'WRITING%20A%20COMPUTER%20PROGRAM%20USING%20A%20SPECIALIZED%20PROGRAMMING%20LANGUAGE'&outFields=*&returnGeometry=false&outSR=4326&f=json")
x = r.json()
programming = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
programming['ISO'] = programming['geoAreaName'].apply(lambda x : dict_map[x])

programming = programming.set_index('ISO')
programming['geoAreaName'] = iso_to_wb['country']
programming = programming.reset_index()
programming = programming[programming.sex_desc=='Both sexes']
programming.to_excel('Data/Social Profile/Population_programming_languages.xlsx')

####  [CO2 emissions per dollar of manufacturing value-ad](https://unstats-undesa.opendata.arcgis.com/datasets/8a0ffe54d84546cda026f03d4ed7b59e/data?geometry=156.851%2C-39.570%2C-88.891%2C62.146) via https://country-profiles.unstatshub.org/pak#goal-8

In [148]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/EN_ATM_CO2MVA_9_4_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()
co2_doll = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
val = ['value_%s'%year for year in range(2000,2018)]
co2_doll = co2_doll[['geoAreaName','parentName',*val]]

co2_doll['ISO'] = co2_doll['geoAreaName'].apply(lambda x : dict_map[x])

co2_doll = co2_doll.set_index('ISO')
co2_doll['geoAreaName'] = iso_to_wb['country']
co2_doll = co2_doll.reset_index()
co2_doll_st = pd.DataFrame(co2_doll.set_index(['geoAreaName','ISO','parentName']).stack()).reset_index()

co2_doll_st.columns = ['Country Name','Country Code','Region','Year','Value']
co2_doll_st['Year'] = co2_doll_st['Year'].apply(lambda x :x[-4:]).astype(int)
co2_doll_st.to_excel('Data/Decarbonization of the power sector/CO2 emissions per dollar of manufacturing value-ad.xlsx')

#### CO2 emissions from intensive sectors - GTAP

In [55]:
gtap_emi = pd.read_excel('Data/GTAP_Emissions and Production Data_2014.xlsx',sheet_name='MDF',skiprows=2)
gtap_emi = gtap_emi.set_index('Country')
gtap_emi['Country Name'] = iso_to_wb['country']
gtap_emi = gtap_emi.dropna(subset=['Country Name'])
gtap_emi = gtap_emi.reset_index()

gtap_emi = gtap_emi.groupby(['Country','Country Name','Commodity']).sum().reset_index()
gtap_emi = gtap_emi.set_index(['Country','Commodity'])

gtap_val = pd.read_excel('Data/GTAP_Emissions and Production Data_2014.xlsx',sheet_name='Production (VOSB)')
gtap_val = gtap_val.set_index(['Country ','Commodity'])

gtap_emi['Production value (M USD)'] = gtap_val['Production value']
gtap_emi = gtap_emi.reset_index()

gtap_emi.to_excel('Data/GTAP_Emission_Intesity_per_sec.xlsx')

#### CO2 emissions from fuel combustion - CPAT

Instructions to update
1. Go to: http://www.oecd-ilibrary.org/energy/data/iea-world-energy-statistics-and-balances_enestats-data-en	
2. Then the data tab next to "Extended world energy balances"	
3. "Customize" -> "Layout" -> switch "Product" to a row category and "Flow" to a column category, take "Time" out of the column categories	
4. Select relevant years, countries and flows	
5. Change the names for China, Russia, Democratic Republic of the Congo, Republic of the Congo, Macedonia, Iran, Kyrgyz Republic, Montenegro, Syria and Vietnam	
6. Use .do file to aggregate into CPAT-format flows and products	


In [592]:
co2_em = pd.read_excel('Data/Decarbonization of the power sector/CO2 emissions from fuel combustion.xlsx',sheet_name='CO2_emissions')

In [594]:
co2_em = co2_em.dropna()
co2_em = co2_em[~co2_em['CPAT country name'].isin(['World','OECD Americas','OECD Asia Oceania','Non-OECD Americas',
        'Europe & Central Asia (excluding high income)','Africa','Middle East','Non-OECD Europe and Eurasia','Kosovo',
        'Non-OECD Asia (excluding China)','World marine bunkers','World aviation bunkers','Curaçao','Hong Kong SAR',
        'Other Africa','Other non-OECD Americas','Other non-OECD Asia','OECD members','Memo: IEA Total','Memo: European Union-28',
                                                  'Memo: FSU 15','Memo: G7','Memo: G8'])]
co2_em['ISO'] = co2_em['CPAT country name'].apply(lambda x : dict_map[x])

In [595]:
co2_em_st = pd.DataFrame(co2_em.set_index(['CPAT country name','ISO','Product']).stack()).reset_index()
co2_em_st.columns = ['Country Name','Country Code','Product','Year','Value']
co2_em_st = co2_em_st.set_index('Country Code')
co2_em_st['Country Name'] = iso_to_wb['country']

co2_em_st['Value'] = co2_em_st['Value'] / 1000
co2_em_st['Unit'] = 'Mt CO2eq'

In [598]:
co2_em_st.reset_index().to_excel('Data/Decarbonization of the power sector/CO2_emissions_fuel_st.xlsx')

#### <font color='green'>Power generation carbon intensity (Draft Power Sector Sustainability ) </font>, <font color='green'>Share of RE (Draft Power Sector Sustainability ) </font>, <font color='green'>Share of variable renewables(Draft Power Sector Sustainability ) </font>
(Need to know how data was produced - CPAT sent by Stephen)

In [43]:
CPAT_IEA_data = pd.read_excel('Data/Decarbonization of the power sector/DraftPowerSectorSustainabilityIndicators.xlsx'
                              ,sheet_name='UnderlyingData')
CPAT_IEA_data = CPAT_IEA_data[~CPAT_IEA_data.Country.isin(['West Bank and Gaza'])]
CPAT_IEA_data['Country Code'] = CPAT_IEA_data.Country.apply(lambda x:dict_map[x])
CPAT_IEA_data = CPAT_IEA_data.set_index('Country Code')
CPAT_IEA_data['Country Name'] = iso_to_wb['country']
CPAT_IEA_data = CPAT_IEA_data.reset_index().drop('Country',axis=1)
CPAT_IEA_data = CPAT_IEA_data[CPAT_IEA_data['CPAT indicator'].isin(['CO2 intensity of power generation','Proportion of renewables in total generation, base year',
                                     'Proportion of variable renewables in total'])]

In [49]:
out = pd.DataFrame()

temp = CPAT_IEA_data[CPAT_IEA_data['CPAT indicator']=='CO2 intensity of power generation']
temp = temp.sort_values('Value2018')

i=1
for val in temp['Value2018'].unique():
    temp.loc[temp['Value2018']==val,'Ranking'] = i
    i = i + 1
out = out.append(temp)

temp = CPAT_IEA_data[CPAT_IEA_data['CPAT indicator']=='Proportion of renewables in total generation, base year']
temp = temp.sort_values('Value2018',ascending=False)

i=1
for val in temp['Value2018'].unique():
    temp.loc[temp['Value2018']==val,'Ranking'] = i
    i = i + 1
out = out.append(temp)

temp = CPAT_IEA_data[CPAT_IEA_data['CPAT indicator']=='Proportion of variable renewables in total']
temp = temp.sort_values('Value2018',ascending=False)

i=1
for val in temp['Value2018'].unique():
    temp.loc[temp['Value2018']==val,'Ranking'] = i
    i = i + 1
out = out.append(temp)
out.to_excel('Data/Decarbonization of the power sector/CO2Em_Intensity_Share_Renewables.xlsx')

#### <font color='red'>Storage Capacity (no country coverage) </font> [Link to IEA Data](https://www.iea.org/reports/energy-storage)

#### [Energy mix](https://www.iea.org/data-and-statistics?country=WORLD&fuel=Energy%20supply&indicator=TPESbySource) (World Energy Balances - IEA)

In [856]:
ener = pd.read_csv('Data/Decarbonization of the power sector/Energy_supply_consumption_IEA.csv')
ener = ener[ener.columns[:-2]]
ener = ener.set_index('COUNTRY')
ener['Country'] = iso_to_wb['country']

ener = ener.dropna().reset_index()

ener_temp = ener[ener.Product.isin(['Geothermal','Solar/wind/other'])]
ener = ener[~ener.Product.isin(['Geothermal','Solar/wind/other'])]

ener_temp = ener_temp.groupby(['COUNTRY','Country','FLOW','Flow','TIME','Time']).sum().reset_index()
ener_temp['Product'] = 'Geothermal/Wind/Solar'
ener = ener.append(ener_temp)
ener.to_excel('Data/Decarbonization of the power sector/Energy_supply_consumption_IEA_ren.xlsx')

#### [Electricity mix](https://www.iea.org/data-and-statistics/data-browser?country=WORLD&fuel=Energy%20supply&indicator=ElecGenByFuel) (World Energy Statistics - IEA)

In [43]:
elec = pd.read_csv('Data/Decarbonization of the power sector/Electricity_mix_IEA.csv')
elec = elec[elec.columns[:-2]]
elec = elec.set_index('COUNTRY')
elec['Country'] = iso_to_wb['country']

elec = elec.dropna().reset_index()
elec = elec[elec['Value']>0]

coal = elec[elec.Product.isin(['Anthracite (kt)','BKB (kt)','Blast furnace gas (TJ-gross)','Coal tar (kt)',
                              'Coke oven coke (kt)','Coke oven gas (TJ-gross)','Coking coal (kt)','Gas works gas (TJ-gross)',
                              'Lignite (kt)','Oil shale and oil sands (kt)','Other bituminous coal (kt)',
                              'Other recovered gases (TJ-gross)','Peat (kt)','Peat products (kt)','Sub-bituminous coal (kt)'])]
coal = coal.groupby(['COUNTRY','Country','Flow','Time']).sum()
coal['Product'] = 'Coal'
coal = coal.reset_index().drop(['TIME'],axis=1)

biofuels = elec[elec.Product.isin(['Biodiesels (kt)', 'Other liquid biofuels (kt)','Primary solid biofuels (TJ-net)'])]
biofuels = biofuels.groupby(['COUNTRY','Country','Flow','Time']).sum()
biofuels['Product'] = 'Biofuels'
biofuels = biofuels.reset_index().drop(['TIME'],axis=1)

oil = elec[elec.Product.isin(['Bitumen (kt)', 'Crude oil (kt)','Fuel oil (kt)','Gas/diesel oil excl. biofuels (kt)',
                             'Kerosene type jet fuel excl. biofuels (kt)','Liquefied petroleum gases (LPG) (kt)',
                             'Motor gasoline excl. biofuels (kt)','Naphtha (kt)','Natural gas liquids (kt)',
                              'Other kerosene (kt)','Other oil products (kt)','Petroleum coke (kt)','Refinery gas (kt)'])]
oil = oil.groupby(['COUNTRY','Country','Flow','Time']).sum()
oil['Product'] = 'Oil'
oil = oil.reset_index().drop(['TIME'],axis=1)

other_sources = elec[elec.Product.isin(['Heat from chemical sources','Other sources'])]
other_sources = other_sources.groupby(['COUNTRY','Country','Flow','Time']).sum()
other_sources['Product'] = 'Other Sources'
other_sources = other_sources.reset_index().drop(['TIME'],axis=1)

remaining_sources = elec[elec.Product.isin(['Hydro','Natural gas (TJ-gross)','Nuclear','Solar photovoltaics',
                                            'Solar thermal (direct use in TJ-net)','Wind','"Tide, wave and ocean"'])]

elec_full = remaining_sources.append(other_sources).append(oil).append(biofuels).append(coal)
elec_full = elec_full.drop(['PRODUCT','FLOW','TIME'],axis=1)

elec_full.columns = ['Country Code','Country Name','Product','Flow','Year','Value']

elec_full.to_excel('Data/Decarbonization of the power sector/Electricity_mix_IEA_v3.xlsx')

#### Share of Renewables

In [52]:
elec = pd.read_excel('Data/Decarbonization of the power sector/Electricity_mix_IEA_v3.xlsx')
elec_last = elec[elec.Year==2018]

elec_last_group = elec_last.groupby(['Country Code','Country Name']).sum().reset_index()
elec_last_group = elec_last_group[['Country Code','Country Name','Value']]

elec_ren = elec_last[elec_last.Product.isin(['Hydro', 'Solar photovoltaics','Wind','Solar thermal (direct use in TJ-net)',
                                            '"Tide, wave and ocean"', 'Other Sources','Biofuels'])]
elec_ren = elec_ren.groupby(['Country Code','Country Name']).sum().reset_index()
elec_ren = elec_ren[['Country Code','Country Name','Value']]

elec_var_ren = elec_last[elec_last.Product.isin(['Solar photovoltaics','Wind','Solar thermal (direct use in TJ-net)',
                                            '"Tide, wave and ocean"', 'Other Sources',])]
elec_var_ren = elec_var_ren.groupby(['Country Code','Country Name']).sum().reset_index()
elec_var_ren = elec_var_ren[['Country Code','Country Name','Value']]

elec_last_group = elec_last_group.set_index('Country Code')
elec_ren = elec_ren.set_index('Country Code')
elec_var_ren = elec_var_ren.set_index('Country Code')

elec_ren['Share Renewables'] = elec_ren['Value'] / elec_last_group['Value']
elec_var_ren['Share Var Renewables'] = elec_var_ren['Value'] / elec_last_group['Value']

elec_ren = elec_ren.sort_values('Share Renewables',ascending=False)
i=1
for val in elec_ren['Share Renewables'].unique():
    elec_ren.loc[elec_ren['Share Renewables']==val,'Ranking'] = i
    i = i + 1

elec_var_ren = elec_var_ren.sort_values('Share Var Renewables',ascending=False)
i=1
for val in elec_var_ren['Share Var Renewables'].unique():
    elec_var_ren.loc[elec_var_ren['Share Var Renewables']==val,'Ranking'] = i
    i = i + 1

elec_ren.to_excel('Data/Decarbonization of the power sector/Share_Renewables.xlsx')
elec_var_ren.to_excel('Data/Decarbonization of the power sector/Share_Var_Renewables.xlsx')

#### IEA objectives Renewables - World Energy Outlook 2020 

In [328]:
ren_IEA = pd.read_excel('Data/Decarbonization of the power sector/IEA_Projection_Electricity.xlsx')

name_regions = pd.read_excel('Country_Regions.xlsx')
name_regions  = name_regions.dropna(subset=['Region'])

In [329]:
NA_countries = ['Canada']
LCR_countries = [a for a in name_regions.loc[name_regions.Region=='Latin America & Caribbean','TableName'].values if a not in ['Brazil']]
europe_countries = [a for a in name_regions.loc[name_regions.Region=='Europe & Central Asia','TableName'].values if a not in ['Armenia','Uzbekistan',\
    'Kazakhstan', 'Kyrgyz Republic','Tajikistan', 'Turkmenistan','Azerbaijan','Georgia','Russian Federation']]
africa_countries = ['Morocco', 'Algeria', 'Tunisia', 'Lybia','Egypt, Arab Rep.',*name_regions.loc[name_regions.Region=='Sub-Saharan Africa','TableName'].values]
ME_countries = ['United Arab Emirates', 'Bahrain', 'Djibouti','Iran, Islamic Rep.', 'Iraq', 'Israel','Jordan', 'Kuwait', 'Lebanon', 'Malta', 'Oman',
       'West Bank and Gaza', 'Qatar', 'Saudi Arabia','Syrian Arab Republic', 'Yemen, Rep.']
eurasia_countries = ['Azerbaijan','Georgia','Kazakhstan', 'Kyrgyz Republic','Tajikistan', 'Turkmenistan','Armenia','Uzbekistan']
asia_pacific_countries = ['Afghanistan','Australia', 'Bangladesh','Korea, Rep.',"Korea, Dem. People's Rep.", 'Mongolia', 'Nepal', 'New Zealand','Pakistan', 'Sri Lanka']
SEA_countries = ['Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao PDR', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam']

In [331]:
df_out = pd.DataFrame()
small_islands = []
for country in name_regions.TableName.unique():
    if country=='United States':
        region = 'United States'
    if country=='Russian Federation':
        region = 'Russia'
    elif country=='China':
        region = 'China'
    elif country=='Japan':
        region = 'Japan'
    elif country=='India':
        region = 'India'
    elif country=='Brazil':
        region = 'Brazil'
    elif country=='South Africa':
        region = 'South Africa'
    elif country=='Canada':
        region = 'North America'
    elif country in LCR_countries:
        region = 'Central and South America'
    elif country in africa_countries:
        region = 'Africa'
    elif country in ME_countries:
        region = 'Middle East'
    elif country in eurasia_countries:
        region = 'Eurasia'
    elif country in asia_pacific_countries:
        region = 'Asia Pacific'
    elif country in SEA_countries:
        region = 'Southeast Asia'
    elif country in europe_countries:
        region = 'Europe'
    else:
        small_islands.append(country)
    
    df_c = ren_IEA[ren_IEA.Region==region]
    df_c['Country Name'] = country 
    df_out = df_out.append(df_c)

df_out = df_out[~df_out['Country Name'].isin(small_islands)]
df_out = df_out.drop(['2019.1','2030.1','2040.1','CAGR 2019-30','CAGR 2019-40'],axis=1)
df_out = pd.DataFrame(df_out.set_index(['Region','Country Name','Type']).stack()).reset_index()
df_out.columns = ['Region_IEA','Country Name','Type','Year','Value']
df_out.to_excel('Data/Decarbonization of the power sector/IEA_Projection_Electricity_stack.xlsx')

<ipython-input-331-ef4d2e463e9c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['Country Name'] = country


#### [Potential impact of climate change on hydropower](https://climateknowledgeportal.worldbank.org/download-data) <font color='red'> Need to define which indicator to use Annual Severe Drought Likelihood ? Period ? RCP? </font>

#### [Energy intensity (OWID)](https://ourworldindata.org/grapher/energy-intensity?tab=chart&country=IND~IDN) can't download it from IEA

In [688]:
ene_int = pd.read_csv('Data/Fuel shift and efficiency in use sectors/energy-intensity.csv')
ene_int = ene_int.set_index('Code')
ene_int['Entity'] = iso_to_wb['country']
ene_int = ene_int.reset_index()
ene_int.columns = ['Country Code','Country Name','Year','Energy consumption per GDP (kWh per $)']
ene_int.to_excel('Data/Fuel shift and efficiency in use sectors/energy_intesity.xlsx')

ene_int = ene_int[ene_int.Year==2016]
ene_int = ene_int.sort_values('Energy consumption per GDP (kWh per $)')
ene_int = ene_int.dropna()
ene_int['Ranking'] = range(1,len(ene_int)+1)
ene_int.to_excel('Data/Fuel shift and efficiency in use sectors/energy_intensity_2016.xlsx')


### [Fossil fuel subsidies - IEA](https://www.iea.org/topics/energy-subsidies)

#### Fossil fuel subsidies (consumption and production) % of GDP,  Fossil fuel subsidies, Fossil fuel subsidies (Average subsidisation rate (%)), Fossil fuel subsidies (Subsidy per capita ($/person)), Fossil fuel subsidies (Total subsidy as share of GDP (%))



In [717]:
foss_sub = pd.read_excel('Data/Fuel shift and efficiency in use sectors/IEA-Fossil-Fuel-Subsidies-2010-2019.xlsx'
                         ,sheet_name='Indicators by country',skiprows=3)

foss_sub['Country Code'] = foss_sub['Country'].apply(lambda x: dict_map[x])
foss_sub = foss_sub.set_index('Country Code')
foss_sub['Country Name'] = iso_to_wb['country']
foss_sub = foss_sub.reset_index().drop('Country',axis=1)

foss_sub_av = foss_sub[['Country Code','Country Name','Average subsidisation rate (%)']]
foss_sub_av = foss_sub_av.sort_values('Average subsidisation rate (%)')
foss_sub_av['Ranking'] = range(1,len(foss_sub_av)+1)
foss_sub_av['Indicator'] = 'Average subsidisation rate (%)'
foss_sub_av.columns = ['Country Code','Country Name','Value','Ranking','Indicator']

foss_sub_cap = foss_sub[['Country Code','Country Name','Subsidy per capita ($/person)']]
foss_sub_cap = foss_sub_cap.sort_values('Subsidy per capita ($/person)')
foss_sub_cap['Ranking'] = range(1,len(foss_sub_cap)+1)
foss_sub_cap['Indicator'] = 'Subsidy per capita ($/person)'
foss_sub_cap.columns = ['Country Code','Country Name','Value','Ranking','Indicator']

foss_sub_GDP = foss_sub[['Country Code','Country Name','Total subsidy as share of GDP (%)']]
foss_sub_GDP = foss_sub_GDP.sort_values('Total subsidy as share of GDP (%)')
foss_sub_GDP['Ranking'] = range(1,len(foss_sub_GDP)+1)
foss_sub_GDP['Indicator'] = 'Total subsidy as share of GDP (%)'
foss_sub_GDP.columns = ['Country Code','Country Name','Value','Ranking','Indicator']

foss_sub_av.append(foss_sub_GDP).append(foss_sub_cap).to_excel('Data/Fuel shift and efficiency in use sectors/IEA_Foss_Subsidies_2019.xlsx')

#### [Fossil-fuel pre-tax subsidies (consumption and production) USD per capita - SDG DATABASE](https://country-profiles.unstatshub.org/pak#goal-8) via https://unstats-undesa.opendata.arcgis.com/datasets/28fbb152ddc74ef4ae6e20e2998f408a?geometry=156.367%2C-37.609%2C-119.610%2C63.296

In [105]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/ER_FFS_PRTSPC_12_c_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()
foss_sub = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
foss_sub = foss_sub[['ISO3','latest_value','max_year']]
foss_sub = foss_sub.sort_values('latest_value')
foss_sub['Ranking'] = range(1,len(foss_sub)+1)
foss_sub = foss_sub.set_index('ISO3')
foss_sub['Country Name'] = iso_to_wb['country']
foss_sub = foss_sub.reset_index()
foss_sub.columns = ['Country Code','Value','Year','Ranking','Country Name']
foss_sub.to_excel('Data/Fuel shift and efficiency in use sectors/Fossil_fuels_pre_tax_sub_cap.xlsx')

### Trade - [WITS](https://wits.worldbank.org/)

Instructions to update : 
1. Go to wits.worldbank.org
2. Login
3. Select Advanced Query and click on Trade Data (UN Comtrade)
4. New query ==> Give query name and description
5. Reporters: Select all countries
6. Products : Nomenclature HS 1988/92,Standard Product Groups Select groups from 01-05_Animal to 90_99_Miscellaneous
7. Partners : Select All
8. Select most recent year
9. Trade Flow : Gross imports & Gross exports
10. Submit

In [ ]:
df = pd.read_csv('Data/Fuel shift and efficiency in use sectors/UN_COMTRADE/DataJobID-2138343_2138343_UNComtrade.csv',engine='python')
countries_reg = pd.read_excel('Country_Regions.xlsx')
countries_reg = countries_reg.set_index('Country Code')
df = df.set_index('PartnerISO3')
df['Region'] = countries_reg['Region']
df['Region_bis'] = df['Region'].copy()
UE = ['Austria','Germany','Belgium','Bulgaria','Cyprus','Croatia','Denmark','Spain','Estonia','Finland',
     'France','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands',
     'Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden']
df = df.reset_index()
df.loc[df[df.PartnerName.isin(UE)].index,'Region_bis'] = 'European Union'
df_group = df.groupby(['ReporterISO3','ReporterName','ProductCode','Year','Region_bis','TradeFlowName']).sum()

In [ ]:
df_group.reset_index().to_excel('Données/Trade_Data/Trade_data_all.xlsx')

#### Share of national income (GDP) spent on fuel imports - WITS x WB

In [738]:
df = pd.read_csv('Data/Fuel shift and efficiency in use sectors/UN_COMTRADE/DataJobID-2138343_2138343_UNComtrade.csv',engine='python')
df = df[df['TradeFlowName']=='Gross Imp.']
df = df[df.ProductCode=='27-27_Fuels']
df = 1000 * df.groupby(['ReporterISO3','ReporterName','Year']).sum()
df = df.reset_index()
df.columns = ['Country Code','Country Name','Year','Temp','TradeValue in USD']
df = df.drop('Temp',axis=1)

temp = df[df.Year==2018]
temp = temp.set_index(['Country Name'])

gdp = wb.download(indicator='NY.GDP.MKTP.CD',country='all',start=1990,end=2020)
gdp = gdp.reset_index().dropna()
gdp.columns = ['Country Name','Year','GDP']
gdp = gdp.set_index(['Country Name'])
gdp = gdp[gdp.Year=='2018']

temp = df[df.Year==2018]
temp.loc[temp['Country Code']=='ZAR','Country Code'] = 'COD'
temp = temp.set_index('Country Code')
temp['Country Name'] = iso_to_wb['country']
temp = temp.reset_index().set_index('Country Name')
temp['GDP'] = gdp['GDP']
temp = temp.dropna()
temp['Share_fuel_imports_GDP'] = temp['TradeValue in USD'] / temp['GDP']

temp = temp.reset_index()

temp_abs = temp[['Country Name','Country Code','TradeValue in USD']]
temp_abs = temp_abs.sort_values('TradeValue in USD')
temp_abs['Ranking'] = range(1,len(temp_abs)+1)
temp_abs['Indicator'] = 'Fuel Imports ($)'
temp_abs.columns = ['Country Name','Country Code','Value','Ranking','Indicator']

temp_GDP = temp[['Country Name','Country Code','Share_fuel_imports_GDP']]
temp_GDP = temp_GDP.sort_values('Share_fuel_imports_GDP')
temp_GDP['Ranking'] = range(1,len(temp_GDP)+1)
temp_GDP['Indicator'] = 'Fuel Imports as share of GDP (%)'
temp_GDP.columns = ['Country Name','Country Code','Value','Ranking','Indicator']

temp_GDP.append(temp_abs).to_excel('Data/Fuel shift and efficiency in use sectors/Share_Fuels_Imports_GDP.xlsx')
# temp.reset_index().to_excel('Data/Fuel shift and efficiency in use sectors/Share_Imports_GDP.xlsx')

C:\Users\Mehdi\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


#### Global Coal Power Capacity - [Download link](https://globalenergymonitor.org/projects/global-coal-plant-tracker/dashboard/)
**Download Dashboard and then Data**

#### Total Operating capacity

In [84]:
cap = pd.read_csv('Data/Decarbonization of the power sector/Total_Capacity.csv')
cap = cap[cap.Country!='Kosovo']
cap['Country Code'] = cap['Country'].apply(lambda x: dict_map[x])

cap = cap.set_index('Country Code')
cap['Country Name'] = iso_to_wb['country']
cap = cap.reset_index().drop('Country',axis=1)

cap.to_excel('Data/Decarbonization of the power sector/Total_Capacity.xlsx')

#### Operating Coal Power by Plant Age and Type


In [155]:
coal_age_op = pd.read_csv('Data/Decarbonization of the power sector/Coal_Plant_type_Plant_type_and_age_data.csv')
coal_age_op = coal_age_op[coal_age_op['Country (Plant type and age)']!='Kosovo']
coal_age_op['Country Code'] = coal_age_op['Country (Plant type and age)'].apply(lambda x:dict_map[x])
coal_age_op = coal_age_op.set_index('Country Code')
coal_age_op['Country Name'] = iso_to_wb['country']
coal_age_op = coal_age_op.reset_index()
coal_age_op.to_excel('Data/Decarbonization of the power sector/Coal_Plant_type_Plant_type_and_age_data.xlsx')

#### Capacity by plant status

In [104]:
cap_stat = pd.read_csv('Data/Decarbonization of the power sector/Capacity_Plant_Status.csv')

In [105]:
cap_stat = cap_stat[cap_stat['Country (2014-2020)']!='Kosovo']
cap_stat['Country Code'] = cap_stat['Country (2014-2020)'].apply(lambda x: dict_map[x])

cap_stat = cap_stat.set_index('Country Code')
cap_stat['Country Name'] = iso_to_wb['country']
cap_stat = cap_stat.reset_index().drop('Country (2014-2020)',axis=1)

cap_stat.to_excel('Data/Decarbonization of the power sector/Capacity_Plant_Status.xlsx')

#### Net Change Coal Capacity

In [110]:
net_change = pd.read_csv('Data/Decarbonization of the power sector/Net_Change_Coal_Capacity.csv')
net_change = net_change[net_change['Country (2000-2020)']!='Kosovo']
net_change['Country Code'] = net_change['Country (2000-2020)'].apply(lambda x: dict_map[x])

net_change = net_change.set_index('Country Code')
net_change['Country Name'] = iso_to_wb['country']
net_change = net_change.reset_index().drop('Country (2000-2020)',axis=1)

net_change.to_excel('Data/Decarbonization of the power sector/Net_Change_Coal_Capacity.xlsx')

#### [Coal Lifetime Emissions](https://globalenergymonitor.org/projects/global-coal-plant-tracker/summary-data/)

In [122]:
coal_em = pd.read_excel('Data/Decarbonization of the power sector/Coal_Lifetime_Emissions.xlsx')

coal_em = coal_em[~coal_em.Country.isin(['Kosovo','Total'])]
coal_em['Country Code'] = coal_em['Country'].apply(lambda x: dict_map[x])

coal_em = coal_em.set_index('Country Code')
coal_em['Country Name'] = iso_to_wb['country']
coal_em = coal_em.reset_index().drop('Country',axis=1)
coal_em = pd.DataFrame(coal_em.set_index(['Country Code','Country Name']).stack()).reset_index()
coal_em.columns = ['Country Code', 'Country Name', 'Type', 'Value']

coal_em.to_excel('Data/Decarbonization of the power sector/Coal_Lifetime_Emissions_stack.xlsx')

#### [Coal Production per country](https://globalenergymonitor.org/projects/global-coal-mine-tracker/summary-data/)

In [134]:
coal_prod = pd.read_excel('Data/Decarbonization of the power sector/Coal_production_country.xlsx')
coal_prod = coal_prod[~coal_prod.Country.isin(['Kosovo','Total'])]

coal_prod['Country Code'] = coal_prod['Country'].apply(lambda x: dict_map[x])

coal_prod = coal_prod.set_index('Country Code')
coal_prod['Country Name'] = iso_to_wb['country']
coal_prod = coal_prod.reset_index().drop('Country',axis=1)
coal_prod = pd.DataFrame(coal_prod.set_index(['Country Code','Country Name']).stack()).reset_index()
coal_prod.columns = ['Country Code', 'Country Name', 'Type', 'Value']

coal_prod.to_excel('Data/Decarbonization of the power sector/Coal_production_country_stack.xlsx')

#### [Proposed Coal Mines GHG emissions](https://globalenergymonitor.org/projects/global-coal-mine-tracker/summary-data/)

In [148]:
ghg_mines_pro = pd.read_excel('Data/Decarbonization of the power sector/GHG_emissions_Proposed_mines.xlsx')

In [149]:
ghg_mines_pro = ghg_mines_pro[~ghg_mines_pro.Country.isin(['Kosovo','Total'])]

ghg_mines_pro['Country Code'] = ghg_mines_pro['Country'].apply(lambda x: dict_map[x])

ghg_mines_pro = ghg_mines_pro.set_index('Country Code')
ghg_mines_pro['Country Name'] = iso_to_wb['country']
ghg_mines_pro = ghg_mines_pro.reset_index().drop('Country',axis=1)
ghg_mines_pro = pd.DataFrame(ghg_mines_pro.set_index(['Country Code','Country Name']).stack()).reset_index()
ghg_mines_pro.columns = ['Country Code', 'Country Name', 'Type', 'Value']

ghg_mines_pro.to_excel('Data/Decarbonization of the power sector/GHG_emissions_Proposed_mines_stack.xlsx')

#### [Operating Coal Mines GHG emissions](https://globalenergymonitor.org/projects/global-coal-mine-tracker/summary-data/)

In [150]:
ghg_mines_op = pd.read_excel('Data/Decarbonization of the power sector/GHG_emissions_Operating_mines.xlsx')

In [151]:
ghg_mines_op = ghg_mines_op[~ghg_mines_op.Country.isin(['Kosovo','Total'])]

ghg_mines_op['Country Code'] = ghg_mines_op['Country'].apply(lambda x: dict_map[x])

ghg_mines_op = ghg_mines_op.set_index('Country Code')
ghg_mines_op['Country Name'] = iso_to_wb['country']
ghg_mines_op = ghg_mines_op.reset_index().drop('Country',axis=1)
ghg_mines_op = pd.DataFrame(ghg_mines_op.set_index(['Country Code','Country Name']).stack()).reset_index()
ghg_mines_op.columns = ['Country Code', 'Country Name', 'Type', 'Value']

ghg_mines_op.to_excel('Data/Decarbonization of the power sector/GHG_emissions_Operating_mines_stack.xlsx')

#### [Gas Pipeline Capacity](https://globalenergymonitor.org/projects/global-fossil-infrastructure-tracker/summary-data/)

In [180]:
gas_pipe = pd.read_excel('Data/Decarbonization of the power sector/Gas_Pipeline_Capacity.xlsx')

In [181]:
gas_pipe = gas_pipe[~gas_pipe.Country.isin(['Scotland','Saint Marino','Kosovo','Total','England','FYROM','Joint Petroleum Development Area'])]

gas_pipe['Country Code'] = gas_pipe['Country'].apply(lambda x: dict_map[x])

gas_pipe = gas_pipe.set_index('Country Code')
gas_pipe['Country Name'] = iso_to_wb['country']
gas_pipe = gas_pipe.reset_index().drop('Country',axis=1)
gas_pipe = pd.DataFrame(gas_pipe.set_index(['Country Code','Country Name']).stack()).reset_index()
gas_pipe.columns = ['Country Code', 'Country Name', 'Type', 'Value']
gas_pipe.to_excel('Data/Decarbonization of the power sector/Gas_Pipeline_Capacity_stack.xlsx')

#### [Oil Pipeline Capacity](https://globalenergymonitor.org/projects/global-fossil-infrastructure-tracker/summary-data/)

In [187]:
oil_pipe = pd.read_excel('Data/Decarbonization of the power sector/Oil_Pipeline_Capacity.xlsx')

oil_pipe = oil_pipe[~oil_pipe.Country.isin(['Disputed Western Sahara/Mauritania','Joint development area Australia/East Timor',
    'Scotland','Saint Marino','Kosovo','Total','England','FYROM','Joint Petroleum Development Area'])]

oil_pipe['Country Code'] = oil_pipe['Country'].apply(lambda x: dict_map[x])

oil_pipe = oil_pipe.set_index('Country Code')
oil_pipe['Country Name'] = iso_to_wb['country']
oil_pipe = oil_pipe.reset_index().drop('Country',axis=1)
oil_pipe = pd.DataFrame(oil_pipe.set_index(['Country Code','Country Name']).stack()).reset_index()
oil_pipe.columns = ['Country Code', 'Country Name', 'Type', 'Value']
oil_pipe.to_excel('Data/Decarbonization of the power sector/Oil_Pipeline_Capacity_stack.xlsx')

#### [LNG Exports Capacity](https://globalenergymonitor.org/projects/global-fossil-infrastructure-tracker/summary-data/)

In [194]:
lng_exp = pd.read_excel('Data/Decarbonization of the power sector/Capacity_LNG_Export.xlsx')

In [195]:
lng_exp = lng_exp[~lng_exp.Country.isin(['Disputed Western Sahara/Mauritania','Joint development area Australia/East Timor',
    'Scotland','Saint Marino','Kosovo','Total','England','FYROM','Joint Petroleum Development Area'])]

lng_exp['Country Code'] = lng_exp['Country'].apply(lambda x: dict_map[x])

lng_exp = lng_exp.set_index('Country Code')
lng_exp['Country Name'] = iso_to_wb['country']
lng_exp = lng_exp.reset_index().drop('Country',axis=1)
lng_exp = pd.DataFrame(lng_exp.set_index(['Country Code','Country Name']).stack()).reset_index()
lng_exp.columns = ['Country Code', 'Country Name', 'Type', 'Value']
lng_exp.to_excel('Data/Decarbonization of the power sector/Capacity_LNG_Export_stack.xlsx')

#### [LNG Imports Capacity](https://globalenergymonitor.org/projects/global-fossil-infrastructure-tracker/summary-data/)

In [197]:
lng_imp = pd.read_excel('Data/Decarbonization of the power sector/Capacity_LNG_Imports.xlsx')

In [198]:
lng_imp = lng_imp[~lng_imp.Country.isin(['Disputed Western Sahara/Mauritania','Joint development area Australia/East Timor',
    'Scotland','Saint Marino','Kosovo','Total','England','FYROM','Joint Petroleum Development Area'])]

lng_imp['Country Code'] = lng_imp['Country'].apply(lambda x: dict_map[x])

lng_imp = lng_imp.set_index('Country Code')
lng_imp['Country Name'] = iso_to_wb['country']
lng_imp = lng_imp.reset_index().drop('Country',axis=1)
lng_imp = pd.DataFrame(lng_imp.set_index(['Country Code','Country Name']).stack()).reset_index()
lng_imp.columns = ['Country Code', 'Country Name', 'Type', 'Value']
lng_imp.to_excel('Data/Decarbonization of the power sector/Capacity_LNG_Imports_stack.xlsx')

#### [Steelmaking Capacity](https://globalenergymonitor.org/projects/global-steel-plant-tracker/dashboard/)

In [109]:
steel = pd.read_excel('Data/Decarbonization of the power sector/Steelmaking Capacity.xlsx')

steel['ISO'] = steel['Country'].apply(lambda x:dict_map[x])
steel = steel.set_index('ISO')
steel['Country Name'] = iso_to_wb['country']
steel.to_excel('Data/Decarbonization of the power sector/Steelmaking Capacity 2.xlsx')

####  <font color='red'>Fuel consumption in transport </font> Missing

####  <font color='red'>Fraction of biofuel in liquid fuel</font> Missing 


####  <font color='red'>Amount of freight transported</font>  [Access](https://data.oecd.org/transport/freight-transport.htm) Only OECD Countries


#### [Freight volume by road transport](https://unstats-undesa.opendata.arcgis.com/datasets/4a5d7189e27148c48f045729ef9e40c8) via https://country-profiles.unstatshub.org/pak#goal-8


In [175]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/IS_RDP_FRGVOL_9_1_2_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()
freight = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
freight = freight[['ISO3','modeOfTransportation_desc','latest_value','max_year']]
freight = freight.set_index('ISO3')
freight['Country Name'] = iso_to_wb['country']

freight.reset_index().to_excel('Data/Fuel shift and efficiency in use sectors/Freight_volume_mode.xlsx')

#### <font color='red'>Mileage per ton of freight transports</font>   Missing global coverage data

#### [Passenger volume by air transport ](https://unstats-undesa.opendata.arcgis.com/datasets/3ed49ad9c9c14694a4b294d01f34d786) via https://country-profiles.unstatshub.org/pak#goal-8


In [176]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/IS_RDP_PFVOL_9_1_2_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

passenger = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
passenger = passenger[['ISO3','modeOfTransportation_desc','latest_value','max_year']]

passenger = passenger.set_index('ISO3')
passenger['Country Name'] = iso_to_wb['country']

passenger.reset_index().to_excel('Data/Fuel shift and efficiency in use sectors/Passenger_volume_mode.xlsx')

#### <font color='red'>Carbon intensity of buildings</font>


#### [Share of the population living in informal settlment](https://unstats-undesa.opendata.arcgis.com/datasets/a21a213e9b4744dd8b53b173ccffd886) via https://country-profiles.unstatshub.org/pak#goal-8 

Warning : some countries have same value 


In [158]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/EN_LND_SLUM_11_1_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

pop_set = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])

In [165]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/EN_LND_SLUM_11_1_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

pop_set = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])

pop_set = pop_set[['ISO3','latest_value','max_year']]
pop_set = pop_set.set_index('ISO3')
pop_set['Country Name'] = iso_to_wb['country']
pop_set = pop_set.sort_values('latest_value')
pop_set = pop_set.dropna()
i=1
for val in pop_set['latest_value'].unique():
    pop_set.loc[pop_set['latest_value']==val,'Ranking'] = i
    i = i + 1

pop_set = pop_set.reset_index()
pop_set.columns = ['Country Code','Value','Year','Country Name','Ranking']
pop_set.to_excel('Data/Social Profile/Population_living_informal_settlement.xlsx')

#### [Energy, post-tax subsidies, Energy, post-tax subsidies, Energy, post-tax subsidies, per capita](https://www.imf.org/~/media/Files/Publications/WP/2019/WPIEA2019089.ashx) 

#### <font color='red'>Carbon intensity of cement</font> Can't find data [here](https://www.iea.org/data-and-statistics/charts/direct-co2-intensity-of-cement-in-the-sustainable-development-scenario-2014-2030)


#### [Proportion of terrestrial key biodiversity areas (KBAs) covered by protected areas](https://unstats-undesa.opendata.arcgis.com/datasets/0e15b1f426814c88b656233c733ac456) via https://country-profiles.unstatshub.org/pak#goal-8

In [180]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/ER_PTD_TERRS_15_1_2_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

ter_KBA = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])

ter_KBA = ter_KBA[['ISO3','latest_value','max_year']]
ter_KBA = ter_KBA.set_index(['ISO3'])
ter_KBA['Country Name'] = iso_to_wb['country']
ter_KBA = ter_KBA.reset_index()
ter_KBA = ter_KBA.dropna()
ter_KBA = ter_KBA.sort_values('latest_value',ascending=False)
i=1
for val in ter_KBA['latest_value'].unique():
    ter_KBA.loc[ter_KBA['latest_value']==val,'Ranking'] = i
    i = i + 1

ter_KBA.to_excel('Data/Food, land-use and natural sink/Ter_KBA_proportion.xlsx')

#### [Proportion of freshwater key biodiversity areas (KBAs) covered by protected areas](https://unstats-undesa.opendata.arcgis.com/datasets/92dd1f2e6b87403cba549c858fa8a913) via https://country-profiles.unstatshub.org/pak#goal-8


In [181]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/ER_PTD_FRWRT_15_1_2_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

fre_KBA = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
fre_KBA = fre_KBA[['ISO3','latest_value','max_year']]
fre_KBA = fre_KBA.set_index(['ISO3'])
fre_KBA['Country Name'] = iso_to_wb['country']
fre_KBA = fre_KBA.reset_index()
fre_KBA = fre_KBA.dropna()
fre_KBA = fre_KBA.sort_values('latest_value',ascending=False)
i=1
for val in fre_KBA['latest_value'].unique():
    fre_KBA.loc[fre_KBA['latest_value']==val,'Ranking'] = i
    i = i + 1

fre_KBA.to_excel('Data/Food, land-use and natural sink/Fresh_KBA_proportion.xlsx')

#### [Share of surface occupied by forest](https://unstats-undesa.opendata.arcgis.com/datasets/4787e933dac34421b4b11427a0b78fbe) via https://country-profiles.unstatshub.org/pak#goal-8 

& Annual change in forest cover (Mean annual growth between 2000-2020)

In [168]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/AG_LND_FRST_15_1_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

forest = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])

forest = pd.read_csv('Data/Food, land-use and natural sink/Forest_area_as_a_proportion_of_total_land_area_percent_.csv')
forest = forest.set_index('ISO3')
forest['Country Name'] = iso_to_wb['country']
forest = forest.reset_index()
evo_forest = forest[['ISO3','Country Name','value_2000', 'value_2010', 'value_2015',
       'value_2016', 'value_2017', 'value_2018', 'value_2019', 'value_2020']]
evo_forest = pd.DataFrame(evo_forest.set_index(['ISO3','Country Name']).stack()).reset_index()
evo_forest.columns = ['ISO3','Country Name','Year','Value']
evo_forest['Year'] = evo_forest['Year'].apply(lambda x :x[-4:]).astype(int)
evo_forest.to_excel('Data/Food, land-use and natural sink/evo_forest.xlsx')

forest = forest[['ISO3','value_2000','value_2020']]
forest['Annual_Change'] = (forest['value_2020'] - forest['value_2000'])/20
forest = forest.dropna()
forest = forest.sort_values('Annual_Change',ascending=False)
forest['Ranking'] = range(1,len(forest)+1)

forest.reset_index().to_excel('Data/Food, land-use and natural sink/forest_cover_change.xlsx')

#### Annual change in agricultural land

In [310]:
share_ag_land = wb.download(indicator='AG.LND.AGRI.ZS',country='all',start=2000,end=2020)
share_ag_land = share_ag_land.reset_index().dropna()
share_ag_land.columns = ['Country Name','Year','Share of ag land (%)']


#### [Yields](http://www.fao.org/faostat/en/#data/QC)
Need to define :

Rice : Rice,paddy or Rice, paddy (milled equivalent)

Maize : Maize or Maize,green

#### [N application](http://www.fao.org/faostat/en/#data/RFN) and [Area Harvested](http://www.fao.org/faostat/en/#data/QC)


In [139]:
n_app = pd.read_csv('Data/Food, land-use and natural sink/N_application.csv')
area_harv = pd.read_csv('Data/Food, land-use and natural sink/Area_harvested.csv')
area_harv = area_harv.groupby(['Area','Unit','Year']).sum()[['Value']].reset_index().set_index('Area')

n_app = n_app[n_app.Year==2018].set_index('Area')
area_harv = area_harv[area_harv.Year==2018]
area_harv['N_application (tons)'] = n_app['Value']

area_harv['N Application per ha'] = 1000 * area_harv['N_application (tons)'] / area_harv['Value']
area_harv['Unit'] = 'kg per ha'

area_harv.columns = ['Unit','Year','Area(ha)','N_application (tons)','N Application per ha']
area_harv = area_harv.reset_index()
area_harv['ISO'] = area_harv['Area'].apply(lambda x: dict_map[x])

area_harv = area_harv.set_index('ISO')
area_harv['Country Name'] = iso_to_wb['country']

area_harv.reset_index().to_excel('Data/Food, land-use and natural sink/N_application_per_ha.xlsx')

#### [Offshore wind potential](https://esmap.org/offshore-wind) 

In [ ]:
wind = pd.read_excel('Data/Decarbonization of the power sector/technicalpotential_countrylevel_wbg_esmap.xlsx')
wind = wind.loc[~pd.isna(wind['Sovereign']), :]
wind['Sovereign'] = wind['Sovereign'].apply(lambda x : map_country(x,dict_map))
wind = wind.set_index('Sovereign')

countries = pd.read_excel('Country_Regions.xlsx')
countries = countries.set_index('TableName')
wind['Territory'] = iso_to_wb['country']
wind = wind.fillna(0)
wind['Total Potential GW'] = wind['Potential Fixed Foundations [GW]'] + wind['Potential Floating Foundations [GW]']
wind = wind.reset_index().groupby(['Sovereign','Territory']).sum()
wind = wind.reset_index().set_index('Sovereign')

pop = pd.read_excel('Data/Social Profile/population.xlsx')
pop = pop.drop(['Indicator Name','Indicator Code'],axis=1)
pop = pd.DataFrame(pop.set_index(['Country Name','Country Code']).stack())
pop = pop.reset_index()
pop.columns = ['Country Name','Country Code','Year','Population']
pop = pop[pop.Year=='2019']
pop = pop.set_index('Country Code')

wind['Population'] = pop['Population']
wind['Potential per capita (kW/cap)'] = 10**6*wind['Total Potential GW'] / wind['Population']
wind = wind.reset_index()

wind_tot = wind[['Sovereign','Territory','Total Potential GW']].sort_values('Total Potential GW',ascending=False)
wind_tot = wind_tot.dropna()
wind_tot['Ranking'] = range(1,len(wind_tot)+1)
wind_tot['Indicator'] = 'Total Potential GW'
wind_tot.columns = ['Country Code','Country Name','Value','Ranking','Indicator']

wind_cap = wind[['Sovereign','Territory','Potential per capita (kW/cap)']].sort_values('Potential per capita (kW/cap)',ascending=False)
wind_cap = wind_cap.dropna()
wind_cap['Ranking'] = range(1,len(wind_cap)+1)
wind_cap['Indicator'] = 'Potential per capita (kW/cap)'
wind_cap.columns = ['Country Code','Country Name','Value','Ranking','Indicator']
wind_cap = wind_cap.append(wind_tot)

wind_cap.to_excel('Data/Decarbonization of the power sector/Wind_Potential_ESMAP.xlsx')

#### [Solar Potential](https://globalsolaratlas.info/map)

In [16]:
solar = pd.read_excel('Data/Decarbonization of the power sector/Solar_Potential.xlsx',sheet_name='Country indicators'
                      ,skiprows=1)
solar = solar.set_index('ISO_A3')
solar['Country Name'] = iso_to_wb['country']
solar = solar.reset_index()
solar = solar.sort_values('Average practical potential \n(PVOUT Level 1, \nkWh/kWp/day), long-term',ascending=False)

i=1
for val in solar['Average practical potential \n(PVOUT Level 1, \nkWh/kWp/day), long-term'].unique():
    solar.loc[solar['Average practical potential \n(PVOUT Level 1, \nkWh/kWp/day), long-term']==val,'Ranking'] = i
    i = i + 1

solar.to_excel('Data/Decarbonization of the power sector/Solar_Potential_v2.xlsx')   

#### [Food Supply Animal vs Vegetal](http://www.fao.org/faostat/en/#data/FBS) (Food Supply)

In [11]:
calory = pd.read_csv(r'Data\Food, land-use and natural sink\Food_Supply_Ani_Vege.csv')
calory['Country Code'] = calory['Area'].apply(lambda x : dict_map[x])
calory = calory.set_index('Country Code')
calory['Country Name'] = iso_to_wb['country']
calory = calory.reset_index()

calory.to_excel('Data/Food, land-use and natural sink/Food_Supply_Ani_Vege.xlsx')

#### [Average supply of protein of animal origin](http://www.fao.org/faostat/en/#data/FS)

In [185]:
prot_an = pd.read_excel('Data/Food, land-use and natural sink/Protein_Animal_Origin.xls')
prot_an['Year'] = prot_an['Year'].apply(lambda x:x[0:4]).astype(int)
prot_an['Country Code'] = prot_an['Area'].apply(lambda x : dict_map[x])
prot_an = prot_an.set_index('Country Code')
prot_an['Country Name'] = iso_to_wb['country']
prot_an = prot_an.reset_index()
prot_an = get_latest_year(prot_an)
prot_an.to_excel('Data/Food, land-use and natural sink/Protein_Animal_Origin.xlsx')

#### [Rice Consumption](http://www.fao.org/faostat/en/#data/SD) g/capita/day

In [197]:
rice = pd.read_excel('Data/Food, land-use and natural sink/Rice_Milled_Quantity.xls')

rice['Country Code'] = rice['Area'].apply(lambda x : dict_map[x])
rice = rice.set_index('Country Code')
rice['Country Name'] = iso_to_wb['country']
rice = rice.reset_index()

rice = get_latest_year(rice)

rice.to_excel('Data/Food, land-use and natural sink/Rice_Milled_food_supply.xlsx')

#### GHG emissions from food systems, by gas - EDGAR Food Dataset

In [134]:
edg = pd.read_excel('Data/Food, land-use and natural sink/EDGAR-FOOD_data.xlsx',
                    sheet_name='Table S7-FOOD emi by sector ',skiprows=2)
edg = edg.set_index('Country_code_A3')
edg['Country Name'] = iso_to_wb['country']
edg = edg.reset_index()
edg = pd.DataFrame(edg.set_index(['Country_code_A3','Name','C_group_IM24_sh','dev_country','Substance','FOOD_system_stage','Country Name']).stack()).reset_index()
edg.columns = ['Country_code_A3','Name','C_group_IM24_sh','dev_country','Substance','FOOD_system_stage','Country Name','Year','Value']
edg['Year'] = edg['Year'].apply(lambda x:x[2:])
edg['Year'] = edg['Year'].astype(int)
edg.reset_index().to_excel('Data/Food, land-use and natural sink/EDGAR_FOOD_data.xlsx')

#### [Proportion of informal employment](https://unstats-undesa.opendata.arcgis.com/datasets/72beed0fca924d31988b49ba180227ba?geometry=-141.813%2C-22.457%2C95.315%2C35.867) via https://country-profiles.unstatshub.org/pak#goal-8

In [183]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/SL_ISV_IFEM_8_3_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

inf_emp = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
inf_emp['Country Code'] = inf_emp['geoAreaName'].apply(lambda x: dict_map[x])
inf_emp = inf_emp.set_index('Country Code')
inf_emp['Country Name'] = iso_to_wb['country']
inf_emp = inf_emp.reset_index()
inf_emp.to_excel('Data/Social Profile/Informal_employment.xlsx')

#### [Food security index](https://foodsecurityindex.eiu.com/Downloads)


In [187]:
food = pd.read_excel('Data/Food, land-use and natural sink/GFSI_2019_Dataset_Scores_260121162046.xlsx')
food = pd.DataFrame(food.set_index('Series').stack()).reset_index()
food.columns = ['Indicator','Country','Value']

for a in food.Indicator.unique():
    temp = food[food.Indicator==a]
    temp = temp.sort_values('Value',ascending=False)
    temp['Rank'] = range(1,len(temp)+1)
    food.loc[temp.index,'Rank'] = temp['Rank']

food['Country code'] = food.Country.apply(lambda x: dict_map[x])
food = food.set_index('Country code')
food['Country'] = iso_to_wb['country']
food.reset_index().to_excel('Data/Food, land-use and natural sink/Food_Security_Index.xlsx')
    

#### [Ambient and indoor air pollution attributable death rate](https://unstats.un.org/sdgs/indicators/database/)

In [219]:
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/SH_AAP_ASMORT_3_9_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

ambient_air = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
ambient_air = ambient_air.sort_values('latest_value')
i=1
for val in ambient_air['latest_value'].unique():
    ambient_air.loc[ambient_air['latest_value']==val,'Ranking'] = i
    i = i + 1
    
r = requests.get("https://services7.arcgis.com/gp50Ao2knMlOM89z/arcgis/rest/services/SH_HAP_ASMORT_3_9_1_2020Q2G03/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")
x = r.json()

household_air = pd.DataFrame.from_dict([x['features'][i]['attributes'] for i in range(len(x['features']))])
household_air = household_air.sort_values('latest_value')
i=1
for val in household_air['latest_value'].unique():
    household_air.loc[household_air['latest_value']==val,'Ranking'] = i
    i = i + 1

pollution = ambient_air.append(household_air)
pollution['Country Code'] = pollution['geoAreaName'].apply(lambda x : dict_map[x])
pollution = pollution.set_index('Country Code')
pollution['Country Name'] = iso_to_wb['country']
pollution = pollution.reset_index()
pollution.to_excel('Data/Managing the transition/Ambient_Indoor_Air_Pollution_stack.xlsx')

#### [Trade_in_environmental_goods](https://climatedata.imf.org/pages/bp-indicators)

In [476]:
trade = pd.read_csv('Data/Efficiency of sectors/Trade_in_environmental_goods.csv')

trade = trade.drop(['ISO2','Code','ObjectId'],axis=1)
trade = trade.set_index('ISO3')
trade['Country'] = iso_to_wb['country']
trade = trade.reset_index()
trade = pd.DataFrame(trade.set_index(['ISO3','Country','Indicator','Unit']).stack()).reset_index()
trade.columns = ['ISO3','Country','Indicator','Unit','Year','Value']
trade = trade[trade.Unit!='US Dollars']
trade['Year'] = trade['Year'].apply(lambda x:x[1:]).astype(int)
trade.to_excel('Data/Efficiency of sectors/Trade_in_environmental_goods_stack.xlsx')

#### [Animal Food supply](http://www.fao.org/faostat/en/#data/CL) & [Crops Food supply](http://www.fao.org/faostat/en/#data/CC)
Food supply(kcal/capita/day)

In [224]:
animal = pd.read_excel('Data/Food, land-use and natural sink/FAOSTAT_Animal_Food_Supply.xls')
animal = animal[~animal.Country.isin(['Netherlands Antilles (former)','Serbia and Montenegro','Sudan (former)'])]
animal['Country Code'] = animal['Country'].apply(lambda x : dict_map[x])
animal = animal.set_index('Country Code')
animal['Country Name'] = iso_to_wb['country']

animal = animal.reset_index()
animal = animal[['Country Code','Country Name','Element','Item','Year','Value']]
animal = animal.groupby(['Country Code','Country Name','Element','Year']).sum().reset_index()
animal['Type'] = 'Livestock and Fish Primary Equivalent'

crops = pd.read_csv('Data/Food, land-use and natural sink/FAOSTAT_Crops_Food_Supply.csv')
crops = crops[~crops.Country.isin(['Netherlands Antilles (former)','Serbia and Montenegro','Sudan (former)'])]
crops['Country Code'] = crops['Country'].apply(lambda x : dict_map[x])
crops = crops.set_index('Country Code')
crops['Country Name'] = iso_to_wb['country']

crops = crops.reset_index()
crops = crops[['Country Code','Country Name','Element','Item','Year','Value']]
crops = crops.groupby(['Country Code','Country Name','Element','Year']).sum().reset_index()
crops['Type'] = 'Crops Primary Equivalent'

food_supply = animal.append(crops)
food_supply.to_excel('Data/Food, land-use and natural sink/Food_supply_Ani_Crops.xlsx')

#### [Undernutrition](http://www.fao.org/faostat/en/#data/FS)

In [179]:
under = pd.read_csv('Data/Food, land-use and natural sink/Nutrition Children.csv')
under = under.rename(columns={'Area':'Country Name'})
under['Country Code'] = under['Country Name'].apply(lambda x:dict_map[x])
under = under.set_index('Country Code')
under['Country Name'] = iso_to_wb['country']
under = under.reset_index()

undernouri = under[under.Item=='Prevalence of undernourishment (percent) (3-year average)']
stun = under[under.Item!='Prevalence of undernourishment (percent) (3-year average)']
stun = get_latest_year(stun)
stun['Value'] = stun['Value'].astype(float)
stun = stun.sort_values('Value',ascending=True)

i=1
for val in stun['Value'].unique():
    stun.loc[stun['Value']==val,'Ranking'] = i
    i = i + 1

undernouri = undernouri.dropna(subset=['Value'])
undernouri = undernouri[undernouri.Value!='<2.5']
undernouri['Year'] = undernouri['Year'].apply(lambda x:x[:4])
undernouri = get_latest_year(undernouri)

undernouri['Value'] = undernouri['Value'].astype(float)
undernouri = undernouri.sort_values('Value',ascending=True)

i=1
for val in undernouri['Value'].unique():
    undernouri.loc[undernouri['Value']==val,'Ranking'] = i
    i = i + 1

undernouri = undernouri.append(stun)
undernouri.to_excel('Data/Food, land-use and natural sink/Nutrition_Children.xlsx')

#### [PRINDEX](https://www.prindex.net/data/)

In [88]:
prindex = pd.read_csv('Data/PRINDEX.csv')
prindex = prindex[~prindex.Name.isin(['Kosovo','Northern Cyprus'])]
prindex['Country Code'] = prindex.Name.apply(lambda x:dict_map[x])
prindex = prindex.set_index('Country Code')
prindex['Country Name'] = iso_to_wb['country']
prindex = prindex.reset_index()
prindex = prindex.drop(['Name','ISO Numeric Code'],axis=1)
prindex = pd.DataFrame(prindex.set_index(['Country Code','Country Name']).stack()).reset_index()
prindex.columns = ['Country Code','Country Name','Indicator','Value']
prindex.to_excel('Data/PRINDEX_Stack.xlsx')

#### CPAT Tool - Contact : Stephen John Stretton

In [156]:
CPAT_macro = pd.read_excel('Data/CCDR-DataPoints-V0.6.xlsx',sheet_name='PresentedTable')
CPAT_macro_g = CPAT_macro[['Country','Scenario','Year','Carbon tax (US$2018/tonCO2)','Change In annual CO2 Emissions - (mtCO2)',
            'Proportional Change in CO2 Emissions Versus Baseline','Net Revenues - (US$bn 2018)','Net Revenues (proportion of GDP)',
            'Air pollution averted deaths','Reduction of air pollution deaths','Direct economic benefits (proportion of GDP)',
            'Direct economic benefits plus cobenefits (US$bn 2018)','Direct economic benefits plus cobenefits (proportion of GDP)']]
CPAT_macro_g = CPAT_macro_g[~CPAT_macro_g.Country.isin(['West Bank and Gaza','Kosovo'])]

CPAT_macro_g['Country Code'] = CPAT_macro_g.Country.apply(lambda x:dict_map[x])
CPAT_macro_g = CPAT_macro_g.set_index('Country Code')
CPAT_macro_g['Country Name'] = iso_to_wb['country']
CPAT_macro_g = CPAT_macro_g.reset_index()
CPAT_macro_g = CPAT_macro_g.drop('Country',axis=1)
CPAT_macro_g.to_excel('Data/CPAT_macro_un.xlsx')

KeyError: "['Direct economic benefits plus cobenefits (proportion of GDP)', 'Direct economic benefits plus cobenefits (US$bn 2018)', 'Proportional Change in CO2 Emissions Versus Baseline', 'Direct economic benefits (proportion of GDP)'] not in index"

In [129]:
CPAT_red = pd.read_excel('Data/CCDR-DataPoints-V0.5_Distribution.xlsx',sheet_name='DistributionalImpactByDecile')
CPAT_red = CPAT_red[CPAT_red['CPAT indicator'].isin(['Relative price impacts on consumption deciles: cash transfer  (% consumption)',
       'Relative price impacts on consumption deciles: direct effect  (% consumption)','Relative price impacts on consumption deciles: indirect effect  (% consumption)',
       'Relative price impacts on consumption deciles: total effect  (% consumption)'])]
CPAT_red = CPAT_red.dropna(subset=['Decile1'])
CPAT_red = CPAT_red[['Country','Scenario','CPAT indicator','Decile1', 'Decile2', 'Decile3', 'Decile4', 'Decile5',
       'Decile6', 'Decile7', 'Decile8', 'Decile9', 'Decile10']]
CPAT_red['Country Code'] = CPAT_red.Country.apply(lambda x:dict_map[x]) 
CPAT_red = CPAT_red.set_index('Country Code')
CPAT_red['Country Name'] = iso_to_wb['country']
CPAT_red = CPAT_red.reset_index()
CPAT_red = CPAT_red.drop('Country',axis=1)
CPAT_red = pd.DataFrame(CPAT_red.set_index(['Country Code','Country Name','Scenario','CPAT indicator']).stack()).reset_index()
CPAT_red.columns = ['Country Code','Country Name','Scenario','CPAT indicator','Decile','Value']
CPAT_red.to_excel('Data/CPAT_decile_red.xlsx')

### Green Finance

#### [Green Bonds](https://climatedata.imf.org/pages/fi-indicators)

In [22]:
green = pd.read_csv('Data/Green Finance/Green_Bonds_IMF.csv')
green = green[green['Country']!='World']
green = green.set_index('ISO3')
green['Country Name'] = iso_to_wb['country']
green = green.reset_index()

green = green.drop(['ObjectId','Country','Code','ISO2'],axis=1)
green = pd.DataFrame(green.set_index(['Country Name','ISO3','Indicator','Unit','Type_of_Issuer','Use_of_Proceed',
                                     'Principal_Currency']).stack()).reset_index()
green.columns = ['Country Name','ISO3','Indicator','Unit','Type_of_Issuer','Use_of_Proceed','Principal_Currency',
                'Year','Value']
green['Year'] = green['Year'].apply(lambda x:x[1:])
green.to_excel('Data/Green Finance/Green_Bonds_IMF.xlsx')

#### [TCFD](https://www.fsb-tcfd.org/supporters/) 

In [31]:
tcfd = pd.read_excel('Data/Green Finance/TCFD.xlsx')
tcfd = tcfd[~tcfd.Location.isin(['Global','Scotland'])]
tcfd['Country Code'] = tcfd.Location.apply(lambda x:dict_map[x])
tcfd = tcfd.set_index('Country Code')
tcfd['Country Name'] = iso_to_wb['country']
tcfd = tcfd.reset_index()
tcfd.to_excel('Data/Green Finance/TCFD_v2.xlsx')

#### [PRI Signatories](https://www.unpri.org/signatories/signatory-resources/signatory-directory)

In [42]:
pri_sign = pd.read_excel('Data/Green Finance/PRI_Signatory_Directory.xlsx')
pri_sign['Country Code'] = pri_sign['HQ Country'].apply(lambda x:dict_map[x])
pri_sign = pri_sign.set_index('Country Code')
pri_sign['Country Name'] = iso_to_wb['country']
pri_sign = pri_sign.reset_index()
pri_sign.to_excel('Data/Green Finance/PRI_Signatory_Directory_v2.xlsx')

#### [UNPRI Regulations](https://www.unpri.org/policy/regulation-database) - Contact within the bank: fstewart1@worldbank.org

In [26]:
pri_reg = pd.read_excel('Data/Green Finance/Public Responsible Investment Database.xlsx',skiprows=1)
pri_reg['Policy'] = ''
for a in ['Corporate ESG disclosure', 'Investor ESG disclosure','Investor ESG Integration', 'Stewardship Code', 'Taxonomy',
       'Sector specific policy','Financial products (green bonds, green labels etc.)', 'Other','National Sustainable Finance Strategy']:
    pri_reg.loc[pri_reg[a]==1,a] = a + ','
    pri_reg[a] = pri_reg[a].fillna('')
    pri_reg['Policy'] = pri_reg['Policy'] + ' ' +  pri_reg[a]

pri_reg = pri_reg.dropna(subset=['Country'])
pri_reg['Country'] = pri_reg['Country'].apply(lambda x:x.rstrip())
pri_reg = pri_reg[~pri_reg.Country.isin(['European Union','Global','ASEAN'])]
pri_reg['Country Code'] = pri_reg.Country.apply(lambda x:dict_map[x])
pri_reg = pri_reg.set_index('Country Code')
pri_reg['Country Name'] = iso_to_wb['country']
pri_reg = pri_reg.reset_index()
pri_reg['Policy'] = pri_reg['Policy'].apply(lambda x:" ".join(x.split()))
pri_reg.loc[pri_reg.Policy=='Investor ESG disclosure, Investor ESG Integration,','Policy'] = 'Investor ESG Integration & Disclosure'
pri_reg.loc[pri_reg.Policy=='Corporate ESG disclosure, Investor ESG disclosure, Investor ESG Integration,','Policy'] = 'Investor & Investor ESG Integration & Disclosure'
pri_reg.loc[pri_reg.Policy=='Corporate ESG disclosure, Investor ESG disclosure,','Policy'] = 'Corporate & Investor ESG Disclosure'
pri_reg.loc[pri_reg.Policy=='Investor ESG disclosure, Investor ESG Integration, National Sustainable Finance Strategy,','Policy'] = 'Investor ESG disclosure & Integration, National Sustainable Finance Strategy,'
pri_reg.loc[pri_reg.Policy=='Investor ESG disclosure, Investor ESG Integration, Stewardship Code,','Policy'] = 'Investor ESG disclosure & Integration, Stewardship Code,'
pri_reg.loc[pri_reg.Policy=='Corporate ESG disclosure, Investor ESG disclosure, Financial products (green bonds, green labels etc.), National Sustainable Finance Strategy,','Policy'] = 'Corporate & Investor ESG disclosure, Financial products (green bonds, green labels etc.), National Sustainable Finance Strategy,'
pri_reg['Policy'] = pri_reg['Policy'].apply(lambda x:x.rstrip(','))
pri_reg.to_excel('Data/Green Finance/PRI_Regulations_v2.xlsx')

#### [ACLED](https://acleddata.com/data-export-tool/) Need to register and create Acess Key  2018 - 2021
[Administrative Boundaries Level 1](http://worldmap.harvard.edu/data/geonode:g2008_1)

In [5]:
import geopandas as gpd

acled = pd.read_csv('Data/ACLED.csv',sep=';')
acled_group = acled.groupby(['iso3','admin2','event_type','sub_event_type','location','latitude','longitude']).count()[['data_id']]
acled_group.columns = ['Number of events']
acled_group = acled_group.reset_index()
gdf = gpd.GeoDataFrame(
    acled_group, geometry=gpd.points_from_xy(acled_group.longitude, acled_group.latitude),crs=4326)
gdf = gdf.set_index('iso3')
gdf['Country Name'] = iso_to_wb['country']
gdf = gdf.reset_index()
gdf['line'] = ['A_%s'%ind for ind in gdf.index]
gdf.to_file('Data/Geo/ACLED1.shp')

#### Employment Multipliers - Source: PURDUE University Contact Person : nlozano@worldbank.org

In [45]:
purdue_demand = pd.read_excel("Data/Social Profile/d_Multipliers.xlsx",sheet_name='Employment_pertype')
purdue_demand = purdue_demand.groupby(['REG','SHOCKED_COMM']).sum()
purdue_demand = purdue_demand.reset_index()

purdue_demand.columns = ['Country Code','Sector','Value']
purdue_demand['Country Code'] = purdue_demand['Country Code'].str.upper()

purdue_CO2 = pd.read_excel("Data/Social Profile/d_CO2emission_FT.xlsx",sheet_name='co2e')
purdue_CO2['Value'] = purdue_CO2['Value'] / 1000

purdue_CO2.columns = ['Country Code','Sector','GHG','Value']
purdue_CO2['Country Code'] = purdue_CO2['Country Code'].str.upper()

purdue_CO2 = purdue_CO2.set_index(['Country Code','Sector'])
purdue_demand = purdue_demand.set_index(['Country Code','Sector'])

purdue_demand['GHG'] = purdue_CO2['Value']

purdue_demand = purdue_demand.reset_index().dropna()
purdue_demand.columns = ['Country Code','Sector','Total Employment','GHG']
purdue_demand = purdue_demand.set_index('Country Code')
purdue_demand['Country Name'] = iso_to_wb['country']
purdue_demand.reset_index().to_excel('Data/Social Profile/Employment_GHG.xlsx')

#### Employment Skills - [Skills Classification](https://ilostat.ilo.org/resources/concepts-and-definitions/classification-occupation/)

In [47]:
purdue_skills = pd.read_excel("Data/Social Profile/d_Multipliers.xlsx",sheet_name='Employment_perskills')

purdue_skills_high = purdue_skills[purdue_skills['LAB'].isin(['tech_aspros','off_mgr_pros'])]
purdue_skills_high = purdue_skills_high.groupby(['REG','SHOCKED_COMM']).sum().reset_index()
purdue_skills_high['LAB'] = 'High Skilled'

purdue_skills_med = purdue_skills[purdue_skills['LAB'].isin(['clerks','service_shop'])]
purdue_skills_med = purdue_skills_med.groupby(['REG','SHOCKED_COMM']).sum().reset_index()
purdue_skills_med['LAB'] = 'Medium Skilled'

purdue_skills_low = purdue_skills[purdue_skills['LAB']=='ag_othlowsk']
purdue_skills_low['LAB'] = 'Low Skilled'

purdue_skills_full = purdue_skills_high.append(purdue_skills_med).append(purdue_skills_low)
purdue_skills_full.columns = ['Country Code','Sector','Value','Skill Level']
purdue_skills_full['Country Code'] = purdue_skills_full['Country Code'].str.upper()
purdue_skills_full = purdue_skills_full.set_index('Country Code')
purdue_skills_full['Country Name'] = iso_to_wb['country']
purdue_skills_full.reset_index().to_excel('Data/Social Profile/Employment_skills.xlsx')

<ipython-input-47-1156467e9b15>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purdue_skills_low['LAB'] = 'Low Skilled'


#### [Total Government Gross Debt](https://data.imf.org/regular.aspx?key=60214266) <font color='red'>Not all countries available</font>


#### [Credit rating](https://countryeconomy.com/ratings) <font color='red'>Painful to download the data
</font>
 